In [4]:
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
import numpy as np
import matplotlib.pylab as plt
import matplotlib
%matplotlib qt
from numpy import linalg as la
from scipy.optimize import fsolve
from scipy import linalg as scpla
# import seaborn as sb
from mpl_toolkits.axes_grid1 import make_axes_locatable
from cmath import *
from mpl_toolkits.mplot3d import Axes3D
from scipy.optimize import fsolve,leastsq 
from math import tanh,cosh
import math
import time

from matplotlib.patches import Ellipse
import matplotlib.transforms as transforms

# from sympy import *
from scipy.linalg import schur, eigvals
from scipy.special import comb, perm

extras_require = {'PLOT':['matplotlib>=1.1.1,<3.0']},

In [5]:
def generate_localstatsmat_eig_desym(Nparams,Grand,rho):
    '''
    Input
      Grand 4, gee,gei,gie,gii
      Nparams: number of E and I neurons (same)
    '''
    NE,NI=Nparams[0],Nparams[1]
    N=NE+NI
    xaxiss,yaxiss =np.arange(NE*2),np.arange(NE*2)
    Xax,Yax=np.meshgrid(xaxiss,yaxiss)
    idxwhere = np.where(Xax>Yax)
    xee,xei,xie,xii=Grand[0],Grand[1],Grand[2],Grand[3]
    X,Xsym,Xasym=np.zeros((N,N)),np.zeros((N,N)),np.zeros((N,N))
    X[:NE,:NE],X[NE:,NE:]=np.random.randn(NE,NE),np.random.randn(NE,NE)
    X[:NE,NE:],X[NE:,:NE]=np.random.randn(NE,NE),np.random.randn(NE,NE)
    XT = X.copy().T

    Xsym = np.sqrt(rho)*X.copy()
    Xsym[idxwhere]=np.sqrt(rho)*XT[idxwhere]
    
    Xasym=np.sqrt(1-rho)*np.random.randn(NE*2,NE*2)
    X=Xsym.copy()+Xasym.copy()
    # components
    Xsym[:NE,:NE],Xsym[NE:,NE:]=xee*Xsym[:NE,:NE]/np.sqrt(NE),xii*Xsym[NE:,NE:]/np.sqrt(NI)
    Xsym[:NE,NE:],Xsym[NE:,:NE]=xei*Xsym[:NE,NE:]/np.sqrt(NE),xie*Xsym[NE:,:NE]/np.sqrt(NI)
    Xasym[:NE,:NE],Xasym[NE:,NE:]=xee*Xasym[:NE,:NE]/np.sqrt(NE),xii*Xasym[NE:,NE:]/np.sqrt(NI)
    Xasym[:NE,NE:],Xasym[NE:,:NE]=xei*Xasym[:NE,NE:]/np.sqrt(NE),xie*Xasym[NE:,:NE]/np.sqrt(NI)
    X=Xsym.copy()+Xasym.copy()
    # theoretical M (grandom) matrix
    gmat = np.array([[xee**2,xei**2],[xie**2,xii**2]])
    # first do not multiply at
    gaverage=0
    for i in range(2):
        for j in range(2):
            gaverage+=gmat[i,j]/2 # ntype=2
    gaverage=np.sqrt(gaverage)
    # for i in range(2):
    #     gmat[:,i]*=at[i]
    eigvgm,eigvecgm=la.eig(gmat)
    
    # properties
    Gamp=np.zeros((2,2))
    Gamp[0,0],Gamp[0,1],Gamp[1,0],Gamp[1,1]=xee,xei,xie,xii
    return (X,Xsym,Xasym,Gamp,eigvgm,eigvecgm,gaverage)

def generate_localstatsmat_eig(Nparams,Grand):
    '''
    Input 
      Grand 4, gee,gei,gie,gii
      Nparams: number of E and I neurons (same)
    '''
    xee,xei,xie,xii=Grand[0],Grand[1],Grand[2],Grand[3]
    NE,NI=Nparams[0],Nparams[1]
    N=NE+NI
    at = Nt/N   
    X=np.zeros((N,N))
    X[:NE,:NE],X[NE:,NE:]=np.random.randn(NE,NE),np.random.randn(NE,NE)
    X[:NE,NE:],X[NE:,:NE]=np.random.randn(NE,NE),np.random.randn(NE,NE)
    X[:NE,:NE],X[NE:,NE:]=xee*X[:NE,:NE]/np.sqrt(NE),xii*X[NE:,NE:]/np.sqrt(NI)
    X[:NE,NE:],X[NE:,:NE]=xei*X[:NE,NE:]/np.sqrt(NE),xie*X[NE:,:NE]/np.sqrt(NI)
    # theoretical M (grandom) matrix 
    gmat = np.array([[xee**2,xei**2],[xie**2,xii**2]])
    # first do not multiply at
    gaverage=0
    for i in range(2):
        for j in range(2):
            gaverage+=gmat[i,j]/2 # ntype=2
    gaverage=np.sqrt(gaverage)
    eigvgm,eigvecgm=la.eig(gmat)
    # properties
    Gamp=np.zeros((2,2))
    Gamp[0,0],Gamp[0,1],Gamp[1,0],Gamp[1,1]=xee,xei,xie,xii

    return (X,Gamp,eigvgm,eigvecgm,gaverage)

def generate_meanmat_eig(Nparams,JEE,JIE,JEI,JII):
    # mean value 
    # first use rank-1 structure
    NE,NI=Nparams[0],Nparams[1]
    N=NE+NI
    at = Nt/N   
    Am=np.zeros((N,N))
    Am[:NE,:NE],Am[:NE,NE:]=JEE/NE,-JEI/NI
    Am[NE:,:NE],Am[NE:,NE:]=JIE/NE,-JII/NI
    # ua,svam,vha=la.svd(Am)
    # va=vha.T
    # svam=svam[0]
    # ua,va=np.zeros((N,N)),np.zeros((N,N))
    # ua[:,0]=1.0/np.sqrt(N)
    # va[:NE,0]=JE/np.sqrt(NE)
    # va[NE:,0]=-JI/np.sqrt(NI)
    # va[:,0]/=-np.linalg.norm(va[:,0])
    # svam=np.sqrt(2*(JE**2+JI**2))
    # vha=va.T

    Jsv=np.zeros((2,2))
    Jsv[0,0],Jsv[0,1],Jsv[1,0],Jsv[1,1]=JEE,JEI,JIE,JII

    return (Am,Jsv)#,ua,svam,va)

# generate gmat, same gavg
def gmatamplitude_eig(gavgfix,typenum):
  Amplit = gavgfix*typenum
  numsample = typenum**2
  Amplitg= np.zeros(numsample)
  idxc=0
  while (1):
    if idxc>=numsample:
      Amplitg[numsample-1]=1.0-np.sum(Amplitg[:numsample-1])
      break
    p=np.random.random(1)
    Amplitg[idxc]=np.minimum(p,1.0-np.sum(Amplitg))
    if np.sum(Amplitg)>1.0:
      continue 
    elif np.sum(Amplitg)==1.0:
      break
    else:
      idxc +=1
      # Amplitg[idxc]=np.min(p,1.0-np.sum(Amplitg))
  # Amplitg=0
  Amplitg*=Amplit
  Amplitg=np.sqrt(Amplitg)
  return Amplitg

def confidence_ellipse(x, y, ax, n_std=3.0, facecolor='none', **kwargs):
    """
    Create a plot of the covariance confidence ellipse of `x` and `y`

    Parameters
    ----------
    x, y : array_like, shape (n, )
        Input data.

    ax : matplotlib.axes.Axes
        The axes object to draw the ellipse into.

    n_std : float
        The number of standard deviations to determine the ellipse's radiuses.

    Returns
    -------
    matplotlib.patches.Ellipse

    Other parameters
    ----------------
    kwargs : `~matplotlib.patches.Patch` properties
    """
    if x.size != y.size:
        raise ValueError("x and y must be the same size")

    cov = np.cov(x, y)
    print(np.shape(cov))
    pearson = cov[0, 1]/np.sqrt(cov[0, 0] * cov[1, 1])
    # Using a special case to obtain the eigenvalues of this
    # two-dimensionl dataset.
    ell_radius_x = np.sqrt(1 + pearson)
    ell_radius_y = np.sqrt(1 - pearson)
    ellipse = Ellipse((0, 0),
        width=ell_radius_x * 2,
        height=ell_radius_y * 2,
        facecolor=facecolor,
        **kwargs)

    # Calculating the stdandard deviation of x from
    # the squareroot of the variance and multiplying
    # with the given number of standard deviations.
    scale_x = np.sqrt(cov[0, 0]) * n_std
    mean_x = np.mean(x)

    # calculating the stdandard deviation of y ...
    scale_y = np.sqrt(cov[1, 1]) * n_std
    mean_y = np.mean(y)

    transf = transforms.Affine2D() \
        .rotate_deg(45) \
        .scale(scale_x, scale_y) \
        .translate(mean_x, mean_y)

    ellipse.set_transform(transf + ax.transData)
    return ax.add_patch(ellipse)

## For Case 1 and 2, without symmetric connectivity
* Based on determinant lemma, calculate eigenvalue outlier (spectrum of eigenvalues)
* Based on perturbation theorem, calculate the statistics of eigenvectors (eigenvector perturbations)
* Validate the results, in the case with homogeneous random gain and the case with heterogeneous random gain 

In [186]:
'''
Cases 1,2 without symmetric connectivity
Focus on standard deviation (variance)
Focus on second moment (self)
'''
# generate mean matrix
nrank=1
Nt=np.array([750,750])
NE,NI=Nt[0],Nt[1]
N=NE+NI
Nparams=np.array([NE,NI])
nrank,ntrial,neta,ngavg=1,20,10,10
## for all casesq
xticks = np.linspace(-4.0,3.0,5)
xlims = [-4.1,3.1]
yticks = np.linspace(-1.3,1.3,3)
ylims = [-1.31,1.31]
theta = np.linspace(0, 2 * np.pi, 200)

''' C0 rules '''
Xtest = np.zeros((NE*2,NE*2))
Xtest[:NE,:NE]=1
idxEE,idyEE=np.where(Xtest>0)
lowinEE=np.where(idxEE<idyEE)
upinEE=np.where(idxEE>idyEE)
idxUpEE,idyUpEE,idxDownEE,idyDownEE=idxEE[lowinEE],idyEE[lowinEE],idxEE[upinEE],idyEE[upinEE]
Xrep = np.zeros_like(Xtest)
Xrep[idxUpEE,idyUpEE]=1
idxUpII,idyUpII,idxDownII,idyDownII=idxUpEE+NE,idyUpEE+NE,idxDownEE+NE,idyDownEE+NE

''' ---------- method 2 ------------------ '''
''' single eta '''
symetaseries = np.linspace(0.1,1.0,10) # a series of symmetricity value
print('symetaseries:',symetaseries)
etaset=0*np.ones(3)
'''
## Three \bar{J} cases
'''
JI,JE,a,b=3.50,1.0,-0.6,0.6 # already recording -3.5# 2.0
a,b=0.0,0.0
# JI,JE,a,b=3.5,1.0,3.5,1.0 # already recording -3.5# 2.0
# JI,JE=0,0
JEE,JIE,JEI,JII=JE+a,JE-a,JI-b,JI+b
'''
Am -- J(g0=0), l0(g0=0), r0(g0=0), S=R1(B1-lambda0In-1)^(-1)L1.T
'''
Am,Jsv=generate_meanmat_eig(Nparams,JEE,JIE,JEI,JII)
# calculate the original \bar{m} and \bar{n}
eigvAm,eigveclAm,eigvecrAm=scpla.eig(Am,left=True,right=True)
meigvecAm=eigvecrAm.copy()
neigvecAm=eigveclAm.copy()
for i in range(nrank):
    normalizecoe=(np.sum(neigvecAm[:,i]*meigvecAm[:,i]))
    neigvecAm[:,i]*=(eigvAm[i]/normalizecoe)
for i in range(1,NE*2):
    normalizecoe=(np.sum(neigvecAm[:,i]*meigvecAm[:,i]))
    neigvecAm[:,i]/=normalizecoe
print('normalization m & n')
print(la.norm(meigvecAm[:,0]),la.norm(neigvecAm[:,0]))
print('eigenvalues:',eigvAm[:nrank])
meanReigAmE,meanReigAmI=np.mean(meigvecAm[:NE,0]),np.mean(meigvecAm[NE:,0])
meanLeigAmE,meanLeigAmI=np.mean(neigvecAm[:NE,0])/eigvAm[0],np.mean(neigvecAm[NE:,0])/eigvAm[0]
'''
for reference
'''
x0,y0=np.reshape(meigvecAm[:,0],(NE*2,1)),np.reshape(neigvecAm[:,0].copy()/eigvAm[0],(NE*2,1))
'''
inverse S
'''
invB1=np.zeros((NE*2-1,NE*2-1))
for i in range(NE*2-1):
    invB1[i,i]=1/(eigvAm[i+1]-eigvAm[0])
X1,Y1=meigvecAm[:,1:].copy(),neigvecAm[:,1:].copy()
SAm = X1@invB1@Y1.T

'''
Network Setting for Iterating Following
'''
signeta=np.ones(3)
# signeta[1]*=(-1)
## heterogeneous random gain
'''
>>>>>>>>>>>>>> Case 1 and Case 2 >>>>>>>>>>>>>>>>>>>>>>>
'''
xee,xei,xie,xii=1.0,1.0,1.0,1.0 ## HOMOgeneous random gain g_kl
# xee,xei,xie,xii=1.0,0.5,0.2,0.8 ## HETEROgeneous random gain
# xee,xei,xie,xii=1.0,0.5,0.3,0.1
'''
Recording Variables
'''
# original data
eigvalseries,mvecseries,nvecseries=np.zeros((ngavg,ntrial,nrank),dtype=complex),np.zeros((ngavg,ntrial,NE*2,nrank),dtype=complex),np.zeros((ngavg,ntrial,NE*2,nrank),dtype=complex)
Reigvecseries,Leigvecseries=np.zeros((ngavg,ntrial,NE*2,N),dtype=complex),np.zeros((ngavg,ntrial,NE*2,N),dtype=complex)
Beigvseries=np.zeros((ngavg,ntrial,N),dtype=complex)
''' All recorded connectivity statistics '''
statesReig,statesLeig=np.zeros((ngavg,ntrial,3,2),dtype=complex),np.zeros((ngavg,ntrial,3,2),dtype=complex) #E,I,Total/mean,std# 
coveigvecseries=np.zeros((ngavg,ntrial,3),dtype=complex) # E, I and total
coveigvecAmseries=np.zeros((ngavg,ntrial,3),dtype=complex)
''' All random matrices for each trials '''
Xprimebasexee=np.zeros((ntrial,NE*2,NE*2)) 
# Y -- identical to X when g=1.0 ([-1])

''' Iterative Processing '''
gaverageseries=np.linspace(0.1,1.0,ngavg)
for iktrial in range(ntrial):
    '''
    for each individual trial, supposed to continuously increase g from zero to one -- see the sensibility changing
    but within one single trial, the random matrix (also random structure) keeps unchanged, thus avoid changing in random pattern
    '''
    # >>> generate the (base/ref) random matrix for each trial
    # Xsym=np.reshape(np.random.randn(NE*2*NE*2),(NE*2,NE*2))/np.sqrt(NE)
    Xsym[:NE,:NE]=np.reshape(np.random.randn(NE*NE),(NE,NE))/np.sqrt(NE)
    Xsym[NE:,NE:]=np.reshape(np.random.randn(NE*NE),(NE,NE))/np.sqrt(NE)
    Xsym[NE:,:NE]=np.reshape(np.random.randn(NE*NE),(NE,NE))/np.sqrt(NE)
    Xsym[:NE,NE:]=np.reshape(np.random.randn(NE*NE),(NE,NE))/np.sqrt(NE)
    XsymT=Xsym.copy().T
    Yidendown=np.reshape(np.random.randn(NE*2*NE*2),(NE*2,NE*2))/np.sqrt(NE)
    Yidenup=np.reshape(np.random.randn(NE*2*NE*2),(NE*2,NE*2))/np.sqrt(NE)
    Xinit = Xsym.copy()
    # >>>>> Now, Xsym and XsymT keep the same while changing the random gain gaverage
    for idxeta,eta in enumerate(gaverageseries):
        #  >>> random gain continuously increases
        etaset=0.0*np.ones(3)#eta*np.ones(3)
        gaverage=eta        
        # ''' heterogeneous ETA method 1 '''
        # ## EE ##
        # asqr=(1+np.sqrt(1-etaset[0]**2))/2.0
        # aamp=np.sqrt(asqr)
        # Xinit[idxDownEE,idyDownEE]=aamp*XsymT[idxDownEE,idyDownEE].copy()+np.sqrt(1-aamp**2)*Xsym[idxDownEE,idyDownEE].copy()
        # Xinit[idxUpEE,idyUpEE]=signeta[0]*aamp*XsymT[idxUpEE,idyUpEE].copy()+signeta[0]*np.sqrt(1-aamp**2)*Xsym[idxUpEE,idyUpEE].copy()
        # ## EI IE##
        # asqr=(1+np.sqrt(1-etaset[1]**2))/2.0
        # aamp=np.sqrt(asqr)
        # Xinit[NE:,:NE]=aamp*XsymT[NE:,:NE].copy()+np.sqrt(1-aamp**2)*Xsym[NE:,:NE].copy()
        # Xinit[:NE,NE:]=signeta[1]*aamp*XsymT[:NE,NE:].copy()+signeta[1]*np.sqrt(1-aamp**2)*Xsym[:NE,NE:].copy()
        # ## II ##
        # asqr=(1+np.sqrt(1-etaset[2]**2))/2.0
        # aamp=np.sqrt(asqr)
        # Xinit[idxDownII,idyDownII]=aamp*XsymT[idxDownII,idyDownII].copy()+np.sqrt(1-aamp**2)*Xsym[idxDownII,idyDownII].copy()
        # Xinit[idxUpII,idyUpII]=signeta[2]*aamp*XsymT[idxUpII,idyUpII].copy()+signeta[2]*np.sqrt(1-aamp**2)*Xsym[idxUpII,idyUpII].copy()
        ''' heterogeneous ETA method 1 '''
        ## EE ##
        asqr=(1-np.sqrt(1-etaset[0]**2))/2.0 
        '''## aamp = 0, XT --> 0 X --> 1'''
        aamp=np.sqrt(asqr) 
        Xinit[:NE,:NE]=signeta[0]*aamp*XsymT[:NE,:NE].copy()+np.sqrt(1-aamp**2)*Xsym[:NE,:NE].copy()
        ## EI IE##
        asqr=(1-np.sqrt(1-etaset[1]**2))/2.0
        aamp=np.sqrt(asqr)
        Xinit[NE:,:NE]=signeta[1]*aamp*XsymT[NE:,:NE].copy()+np.sqrt(1-aamp**2)*Xsym[NE:,:NE].copy()
        Xinit[:NE,NE:]=signeta[1]*aamp*XsymT[:NE,NE:].copy()+np.sqrt(1-aamp**2)*Xsym[:NE,NE:].copy()
        ## II ##
        asqr=(1-np.sqrt(1-etaset[2]**2))/2.0
        aamp=np.sqrt(asqr)
        Xinit[NE:,NE:]=signeta[2]*aamp*XsymT[NE:,NE:].copy()+np.sqrt(1-aamp**2)*Xsym[NE:,NE:].copy()
        # ''' HETEROGENEOUS ETA method 2 '''
        # # EE#
        # Xinit[idxDownEE,idyDownEE]=Xsym[idxDownEE,idyDownEE]*np.sqrt(etaset[0])+Yidendown[idxDownEE,idyDownEE].copy()*np.sqrt(1-etaset[0])
        # Xinit[idxUpEE,idyUpEE]=signeta[0]*XsymT[idxUpEE,idyUpEE]*np.sqrt(etaset[0])+Yidenup[idxUpEE,idyUpEE].copy()*np.sqrt(1-etaset[0])
        # # EI/IE#
        # Xinit[NE:,:NE]=Xsym[NE:,:NE]*np.sqrt(etaset[1])+Yidendown[NE:,:NE].copy()*np.sqrt(1-etaset[1])
        # Xinit[:NE,NE:]=signeta[1]*XsymT[:NE,NE:]*np.sqrt(etaset[1])+Yidenup[:NE,NE:].copy()*np.sqrt(1-etaset[1])
        # # EE#
        # Xinit[idxDownII,idyDownII]=Xsym[idxDownII,idyDownII]*np.sqrt(etaset[2])+Yidendown[idxDownII,idyDownII].copy()*np.sqrt(1-etaset[2])
        # Xinit[idxUpII,idyUpII]=signeta[2]*XsymT[idxUpII,idyUpII]*np.sqrt(etaset[2])+Yidenup[idxUpII,idyUpII].copy()*np.sqrt(1-etaset[2])
        ''' THEORETICAL FULL ETA '''
        symtheofull=gaverage**2*(etaset[0]*signeta[0]+etaset[2]*signeta[2]+2*etaset[1]*signeta[1])/4.0#+(a-b)**2/(NE*2)
        # ''' heterogeneous '''
        X=Xinit.copy()
        X[:NE,:NE]*=(xee*gaverage/np.sqrt(2))
        X[:NE,NE:]*=(xei*gaverage/np.sqrt(2))
        X[NE:,:NE]*=(xie*gaverage/np.sqrt(2))
        X[NE:,NE:]*=(xii*gaverage/np.sqrt(2))

        '''
        >> start perturbed matrix
        >> * perturbed matrix: J; low rank structures: Am; random components: X
        >> * eigenvectors perturbed matrix: Reig, Leig --> renormalized by x0,y0,--> get xnorm0,ynorm0
        >> * low rank structure: x0,y0, 

        >>> statistics (recording)
        >> * mean and standard deviation of elements in eigenvectors -- E and I populations (individual)
        >> * overlap and tilting? between left and right eigenvectors --> relate to eigenvalue
        '''
        # overall
        J = X.copy()+Am.copy()
        JT = J.copy().T
        symnumfull=np.sum(np.sum(J*JT))/(4*NE**2)-np.mean(np.mean(J.copy()))**2
        symnumfull*=2*NE
        print('eta full:',symtheofull,'; num:',symnumfull)
        '''
        statistics >>>>>>>>>>> Full connectivity
        '''
        # properties
        eigvJ,eigrvecJ=la.eig(J)
        inveigrvecJ=la.inv(eigrvecJ)
        meig = np.squeeze(eigrvecJ[:,:].copy())
        # meig = np.reshape(meig,(NE*2,nrank))
        neig = np.squeeze(inveigrvecJ[:,:].copy()) # inverse
        neig=neig.copy().T
        # neig = np.reshape(neig.T,(NE*2,nrank))
        for i in range(nrank):
            neig[:,i]*=eigvJ[i]
        for j in range(1,N):
            neig[:,j]*=eigvJ[j]
        ''' low rank properties m,n,eigenvalues '''
        eigvalseries[idxeta,iktrial,0],mvecseries[idxeta,iktrial,:,0],nvecseries[idxeta,iktrial,:,0]=eigvJ[0],meig[:,0].copy(),neig[:,0].copy() 
        
        Leig, Reig = np.zeros((N,N)),np.zeros((N,N))
        ''' The first '''
        Reig=meig.copy()
        normval=np.sum(Reig*neig.copy(),axis=0)
        normval=np.repeat(np.reshape(normval,(1,N)),N,axis=0)
        ''' eigendecomposition properties left and right eigenvectors '''
        Leig=neig.copy()/normval.copy() 
        '''
        normalization 0 (reference)
        '''
        tildex,tildey=np.reshape(Reig[:,0].copy(),(NE*2,1)),np.reshape(Leig[:,0].copy(),(NE*2,1))
        if (np.mean(tildex[:NE])*x0[0,0])<0:
            tildex*=(-1)
            tildey*=(-1)
        xnorm0=np.sqrt(np.squeeze(tildey.T@x0)/np.squeeze(y0.T@tildex))*tildex
        ynorm0=np.sqrt(np.squeeze(tildex.T@y0)/np.squeeze(x0.T@tildey))*tildey

        Reigvecseries[idxeta,iktrial,:,:],Leigvecseries[idxeta,iktrial,:,:]=Reig.copy(),Leig.copy()
        Beigvseries[idxeta,iktrial,:]=eigvJ.copy()
        Reigvecseries[idxeta,iktrial,:,0],Leigvecseries[idxeta,iktrial,:,0]=xnorm0[:,0].copy(),ynorm0[:,0].copy()

        '''  RECORDING STATES 
        * statesReig/Leig[grandom,trial,E/I/Total,mean/std]

        '''
        statesReig[idxeta,iktrial,0,0],statesReig[idxeta,iktrial,1,0]=np.mean(Reigvecseries[idxeta,iktrial,:NE,0]),np.mean(Reigvecseries[idxeta,iktrial,NE:,0])
        statesReig[idxeta,iktrial,2,0]=np.mean(Reigvecseries[idxeta,iktrial,:,0])
        statesReig[idxeta,iktrial,0,1],statesReig[idxeta,iktrial,1,1]=np.std(Reigvecseries[idxeta,iktrial,:NE,0]),np.std(Reigvecseries[idxeta,iktrial,NE:,0])
        statesReig[idxeta,iktrial,2,1]=np.std(Reigvecseries[idxeta,iktrial,:,0])
       
        statesLeig[idxeta,iktrial,0,0],statesLeig[idxeta,iktrial,1,0]=np.mean(Leigvecseries[idxeta,iktrial,:NE,0]),np.mean(Leigvecseries[idxeta,iktrial,NE:,0])
        statesLeig[idxeta,iktrial,2,0]=np.mean(Leigvecseries[idxeta,iktrial,:,0])
        statesLeig[idxeta,iktrial,0,1],statesLeig[idxeta,iktrial,1,1]=np.std(Leigvecseries[idxeta,iktrial,:NE,0]),np.std(Leigvecseries[idxeta,iktrial,NE:,0])
        statesLeig[idxeta,iktrial,2,1]=np.std(Leigvecseries[idxeta,iktrial,:,0])

        ''' covariance '''
        '''
        Note, we should change the normalization for the first item to the (Refer) normalization 
        tildeReig,tildeLeig
        normalization 0
        '''
        ## normalization 0 reference
        tildeReig,tildeLeig=np.squeeze(Reigvecseries[idxeta,iktrial,:,:]),np.squeeze(Leigvecseries[idxeta,iktrial,:,:])
        reig,leig=np.reshape(tildeReig[:,0].copy(),(NE*2,1)),np.reshape(tildeLeig[:,0].copy(),(NE*2,1))
        ## mean at eact point
        rEceneig,rIceneig = np.mean(reig[:NE,0]),np.mean(reig[NE:,0])
        lEceneig,lIceneig = np.mean(leig[:NE,0]),np.mean(leig[NE:,0])
        ## random components
        rrandeig,lrandeig=reig.copy(),leig.copy()
        rrandeig[:NE,0],rrandeig[NE:,0]=rrandeig[:NE,0]-rEceneig,rrandeig[NE:,0]-rIceneig
        lrandeig[:NE,0],lrandeig[NE:,0]=lrandeig[:NE,0]-lEceneig,lrandeig[NE:,0]-lIceneig

        coveigvecseries[idxeta,iktrial,0]=np.reshape(rrandeig[:NE,0],(NE,1)).T@np.reshape(lrandeig[:NE,0],(NE,1))
        coveigvecseries[idxeta,iktrial,1]=np.reshape(rrandeig[NE:,0],(NE,1)).T@np.reshape(lrandeig[NE:,0],(NE,1))
        coveigvecseries[idxeta,iktrial,2]=coveigvecseries[idxeta,iktrial,0]+coveigvecseries[idxeta,iktrial,1]
        coveigvecAmseries[idxeta,iktrial,0]=np.reshape(reig[:NE,0]-meanReigAmE,(NE,1)).T@np.reshape(leig[:NE,0]-meanLeigAmE,(NE,1))
        coveigvecAmseries[idxeta,iktrial,1]=np.reshape(reig[NE:,0]-meanReigAmI,(NE,1)).T@np.reshape(leig[NE:,0]-meanLeigAmI,(NE,1))
        coveigvecAmseries[idxeta,iktrial,2]=coveigvecAmseries[idxeta,iktrial,0]+coveigvecAmseries[idxeta,iktrial,1]

        ''' record the random derivative '''
        if idxeta == ngavg-1:
            print('recording!!!')
            Xprimebasexee[iktrial,:,:]=X.copy()


symetaseries: [0.1 0.2 0.3 0.4 0.5 0.6 0.7 0.8 0.9 1. ]
normalization m & n
1.000000000000003 5.147815070493495
eigenvalues: [-2.5+0.j]
<ipython-input-186-55e815eb8708>:72: ComplexWarning: Casting complex values to real discards the imaginary part
  invB1[i,i]=1/(eigvAm[i+1]-eigvAm[0])
eta full: 0.0 ; num: -7.130284721122621e-06
eta full: 0.0 ; num: -1.626423564187259e-05
eta full: 0.0 ; num: -2.7401852762208623e-05
eta full: 0.0 ; num: -4.0543136082159296e-05
eta full: 0.0 ; num: -5.568808560171826e-05
eta full: 0.0 ; num: -7.283670132087218e-05
eta full: 0.0 ; num: -9.19889832396452e-05
eta full: 0.0 ; num: -0.00011314493135802269
eta full: 0.0 ; num: -0.00013630454567600464
eta full: 0.0 ; num: -0.00016146782619358982
recording!!!
eta full: 0.0 ; num: 7.502085036861366e-06
eta full: 0.0 ; num: 1.7478245179585816e-05
eta full: 0.0 ; num: 2.9928480428177162e-05
eta full: 0.0 ; num: 4.4852790782640484e-05
eta full: 0.0 ; num: 6.225117624297388e-05
eta full: 0.0 ; num: 8.212363680916973

In [233]:
# saving data
now = time.strftime("%Y-%m-%d-%H_%M_%S",time.localtime(time.time()))
Jparameters = np.zeros(4+1)
Jparameters[0],Jparameters[1],Jparameters[2],Jparameters[3]=JEE,JEI,JIE,JII
Jparameters[4]=NE
Randgain=np.zeros(4)
Randgain[0],Randgain[1],Randgain[2],Randgain[3]=xee,xei,xie,xii

np.savez(now+'_perturbationHomoZeroeta_data',x0=x0,y0=y0,Jparameters=Jparameters,Randgain=Randgain,gaverageseries=gaverageseries,etaset=etaset,signeta=signeta,eigvalseries=eigvalseries,Reigvecseries=Reigvecseries[:,:,:,0],Leigvecseries=Leigvecseries[:,:,:,0],Beigvseries=Beigvseries[:,:,0],meigvecAm=meigvecAm[:,0],neigvecAm=neigvecAm[:,0],eigvAm=eigvAm,statesReig=statesReig,statesLeig=statesLeig,coveigvecseries=coveigvecseries,coveigvecAmseries=coveigvecAmseries,Xprimebasexee=Xprimebasexee)

In [183]:
'''
without symmetry, 
eigvAm
'''
idxgavg,idxtrial=9,8
# figtspt,axtspt=plt.subplots(figsize=(6,3))
axtspt.scatter(np.real(Beigvseries[idxgavg,idxtrial,1:]),np.imag(Beigvseries[idxgavg,idxtrial,1:]),s=5,c='green',alpha=0.125)  # >>>>>>>>>>>>>>>>>>>.
axtspt.scatter(np.real(Beigvseries[idxgavg,:,0]),np.imag(Beigvseries[idxgavg,:,0])-0.2,s=20,c='orange',marker='^',alpha=0.5)  # >>>>>>>>>>>>>>>>>>>
axtspt.set_aspect('equal')
axtspt.plot(np.ones(2)*eigvAm[0],np.array([-1.0,1.0]),linestyle='--',linewidth=1.5,color='grey')
axtspt.spines['right'].set_color('none')
axtspt.spines['top'].set_color('none')
#设置x坐标刻度数字或名称的位置
axtspt.xaxis.set_ticks_position('bottom')
#设置边框位置
axtspt.spines['bottom'].set_position(('data', 0))

aee,aei,aie,aii=xee,xei,xie,xii
print(xee,xei,xie,xii)
idxtrial=10
eta=0
xticks = np.linspace(-1.3,1.3,2)
xlims = [-1.3,1.3]
yticks = np.linspace(-1.3,1.3,2)
ylims = [-1.31,1.31]
theta = np.linspace(0, 2 * np.pi, 200)
# first do not multiply at
ahomo=gaverageseries[idxgavg]
xee_,xei_,xie_,xii_=ahomo*aee/np.sqrt(2),ahomo*aei/np.sqrt(2),ahomo*aie/np.sqrt(2),ahomo*aii/np.sqrt(2)
gmat = np.array([[xee_**2,xei_**2],[xie_**2,xii_**2]])
gaverage_=0
for i in range(2):
    for j in range(2):
        gaverage_+=gmat[i,j]/2 # ntype=2
gaverage_=np.sqrt(gaverage_)
eigvgm,eigvecgm=la.eig(gmat) 
r_g2=np.max(eigvgm)
r_g = np.sqrt(r_g2)
print(r_g)
eta=0
longaxis,shortaxis=(1+eta)*r_g ,(1-eta)*r_g 
xr = longaxis*np.cos(theta)
yr = shortaxis*np.sin(theta)
axtspt.plot(xr, yr, color="darkorange", linewidth=2.5,linestyle='--',label=r'ellipse') # >>>>>>>>>>>>>
avgg=np.sum(np.sum(gmat))*2/4.0
avgg=np.sqrt(avgg)
xa = avgg*np.cos(theta)
ya = avgg*np.sin(theta)
axtspt.plot(xa, ya, color="gray", linewidth=2.5,linestyle='--',label=r'ellipse')
print(avgg,gaverage_)



1.0 0.5 0.3 0.1
0.35746121983492835
0.2904737509655563 0.2904737509655563
/Applications/Anaconda/anaconda3/lib/python3.8/site-packages/numpy/core/_asarray.py:85: ComplexWarning: Casting complex values to real discards the imaginary part
  return array(a, dtype, copy=False, order=order)


## individual perturbations of left and right eigenvectors
* calculate the predicted perturbations (theo)
* comparing with numerical perturbations (num)

In [8]:
'''
>>>>>>>>>>>>>>>>> Compare the individual response (not statistics), NOTE:
To make direct comparison between numerical and perturbation theorem, keep the same random matrix for each trial
choose one trial -- determine the random derivative -- calculate the individual elements in vector

'''

'''
# loading data
# strfile='2021-03-13-15_20_04_perturbationHomoZeroeta_data'# homogeneous random gain
strfile='2021-03-13-15_10_37_perturbationHeterZeroeta_data' # heterogeneous random gain
data = np.load(strfile+'.npz')
gaverageseries = data['gaverageseries']
x0,y0=data['x0'],data['y0']
meigvecAm,neigvecAm=data['meigvecAm'],data['neigvecAm']
# perturbed
Leigvecseries,Reigvecseries=data['Leigvecseries'],data['Reigvecseries']
Xprimebasexee=data['Xprimebasexee']
eigvAm=data['eigvAm']
ngavg,ntrial,N=np.shape(Leigvecseries)
Leigvecseries=np.expand_dims(Leigvecseries,axis=3)
Reigvecseries=np.expand_dims(Reigvecseries,axis=3)
NE=int(N/2.0)
'''

'''
calculate S0
'''
xticks = np.linspace(-0.08,0.08,2)
xlims = [-0.08,0.08]
yticks = np.linspace(-0.08,0.08,2)
ylims = [-0.08,0.08]
Z = np.zeros((NE*2-1,NE*2-1))
for i in range(NE*2-1):
    b1=eigvAm[i+1]
    Z[i,i]=1/(-eigvAm[0])
S0=np.zeros((NE*2,NE*2))
R1,L1=np.squeeze(meigvecAm[:,1:]),np.squeeze(neigvecAm[:,1:])
S0=R1@Z@L1.T

'''
# CHOOSE ONE TRIAL
'''
idxtrial,idxtrial_=8,3
fig,ax=plt.subplots(2,3,figsize=(6,4))
Xprime = np.squeeze(Xprimebasexee[idxtrial,:,:])
idxgsamples=np.array([1,5,9])
gsamples=gaverageseries[idxgsamples]
rperturb_num,lperturb_num = np.zeros((NE*2,3)),np.zeros((NE*2,3))
rperturb_the,lperturb_the = np.zeros((NE*2,3)),np.zeros((NE*2,3))
rprime_the,lprime_the = np.squeeze(Xprime@x0/eigvAm[0]),np.squeeze(Xprime.T@y0/eigvAm[0])
for i in range(len(idxgsamples)):
    lperturb_num[:,i] = np.squeeze(Leigvecseries[idxgsamples[i],idxtrial,:,0])-np.squeeze(y0)
    rperturb_num[:,i] = np.squeeze(Reigvecseries[idxgsamples[i],idxtrial,:,0])-np.squeeze(x0)

    lperturb_the[:,i] = lprime_the*gsamples[i]
    rperturb_the[:,i] = rprime_the*gsamples[i]
    ax[0,i].plot(xticks,yticks,color='darkred',linestyle='--')
    ax[1,i].plot(xticks,yticks,color='darkred',linestyle='--')
    ax[0,i].scatter(np.real(rperturb_the[:,i]),rperturb_num[:,i],s=5,c='gray',alpha=0.5)
    ax[1,i].scatter(np.real(lperturb_the[:,i]),lperturb_num[:,i],s=5,c='gray',alpha=0.5)
    ax[0,i].set_xticks([])
    ax[0,i].set_yticks([])
    ax[1,i].set_xticks([])
    ax[1,i].set_yticks([])

for i in range(3):
    ax[0,i].set_xlim(xlims)
    ax[0,i].set_ylim(ylims)
    ax[1,i].set_xlim(xlims)
    ax[1,i].set_ylim(ylims)
ax[0,0].set_yticks(yticks)
ax[1,0].set_yticks(yticks)
for i in range(3):
    ax[1,i].set_xticks(xticks)
# print(np.shape(rprime_the),np.shape(rprime_num))
# ax.scatter(np.real(rprime_the),rprime_num,c='gray',alpha=0.5)
# ax.set_xlim([-0.05,0.05])
# ax.set_ylim([-0.05,0.05])
'''
# CHOOSE TWO TRIALS
'''
idxtrial,idxtrial_=8,3
fig,ax=plt.subplots(2,3,figsize=(6,4))
Xprime = np.squeeze(Xprimebasexee[idxtrial,:,:])
idxgsamples=np.array([1,5,9])
gsamples=gaverageseries[idxgsamples]
rperturb_num,lperturb_num = np.zeros((NE*2,3)),np.zeros((NE*2,3))
rperturb_the,lperturb_the = np.zeros((NE*2,3)),np.zeros((NE*2,3))
rprime_the,lprime_the = np.squeeze(Xprime@x0/eigvAm[0]),np.squeeze(Xprime.T@y0/eigvAm[0])
for i in range(len(idxgsamples)):
    lperturb_num[:,i] = np.squeeze(Leigvecseries[idxgsamples[i],idxtrial_,:,0])-np.squeeze(y0)
    rperturb_num[:,i] = np.squeeze(Reigvecseries[idxgsamples[i],idxtrial_,:,0])-np.squeeze(x0)

    lperturb_the[:,i] = lprime_the*gsamples[i]
    rperturb_the[:,i] = rprime_the*gsamples[i]
    ax[0,i].plot(xticks,yticks,color='darkred',linestyle='--')
    ax[1,i].plot(xticks,yticks,color='darkred',linestyle='--')
    ax[0,i].scatter(np.real(rperturb_the[:,i]),rperturb_num[:,i],s=5,c='gray',alpha=0.5)
    ax[1,i].scatter(np.real(lperturb_the[:,i]),lperturb_num[:,i],s=5,c='gray',alpha=0.5)
    ax[0,i].set_xticks([])
    ax[0,i].set_yticks([])
    ax[1,i].set_xticks([])
    ax[1,i].set_yticks([])



for i in range(3):
    ax[0,i].set_xlim(xlims)
    ax[0,i].set_ylim(ylims)
    ax[1,i].set_xlim(xlims)
    ax[1,i].set_ylim(ylims)
ax[0,0].set_yticks(yticks)
ax[1,0].set_yticks(yticks)
for i in range(3):
    ax[1,i].set_xticks(xticks)
# print(np.shape(rprime_the),np.shape(rprime_num))
# ax.scatter(np.real(rprime_the),rprime_num,c='gray',alpha=0.5)
# ax.set_xlim([-0.05,0.05])
# ax.set_ylim([-0.05,0.05])

<ipython-input-8-68ae079c047d>:36: ComplexWarning: Casting complex values to real discards the imaginary part
  Z[i,i]=1/(-eigvAm[0])
<ipython-input-8-68ae079c047d>:53: ComplexWarning: Casting complex values to real discards the imaginary part
  lperturb_num[:,i] = np.squeeze(Leigvecseries[idxgsamples[i],idxtrial,:,0])-np.squeeze(y0)
<ipython-input-8-68ae079c047d>:54: ComplexWarning: Casting complex values to real discards the imaginary part
  rperturb_num[:,i] = np.squeeze(Reigvecseries[idxgsamples[i],idxtrial,:,0])-np.squeeze(x0)
<ipython-input-8-68ae079c047d>:56: ComplexWarning: Casting complex values to real discards the imaginary part
  lperturb_the[:,i] = lprime_the*gsamples[i]
<ipython-input-8-68ae079c047d>:57: ComplexWarning: Casting complex values to real discards the imaginary part
  rperturb_the[:,i] = rprime_the*gsamples[i]
<ipython-input-8-68ae079c047d>:53: ComplexWarning: Casting complex values to real discards the imaginary part
  lperturb_num[:,i] = np.squeeze(Leigvecse

## population statistics 
* first determint lemma for eigenvalue outlier -- no change
* perturbation theorem for variance of eigenvalue
* perturbation theorem for variance of left and right eigenvectors


In [21]:
'''
### >>>>>>>>>>>>>>>>> Statistics Linearly Change >>>>>>>>>>>>>>>>
Case 1,2 focus on second-order statstics WITHIN (left/right) eigenvectors
Calculate the statistics of eigenvalue, eigenvectors
  * mean, variance of the perturbation of eigenvalue \mu(derivative of lambda),\sigma(derivative of lambda)
'''
'''
Variables:
    std_perturb_Lambda_num (1)
    std_perturb_Lambda_theo (1)
    std_perturbLambda_linear: reconstruct np.zeros(ngavg)
    std_perturbLambda_linear_theo=std_perturb_Lambda_theo*gaverageseries
'''
## calculating the statistics of perturbation lambda
# Coding uses only one point [linear assumption]
std_perturb_Lambda_numSE  = np.std(eigvalseries[ngavg-1,:,0])
std_perturb_Lambda_theo = np.sqrt((JE**2*(xee**2+xei**2)+JI**2*(xie**2+xii**2))/(2*NE)/(JE-JI)**2)
## theoretical curve for standard deviation of the eigenvalue
std_perturbLambda_linear_theo=std_perturb_Lambda_theo*gaverageseries
std_perturbLambda_linear_numSE=std_perturb_Lambda_numSE*gaverageseries
'''
Variance of the perturbations in eigenvalue lambda
\delta lambda =\Delta lambda = (1) unit change in g
'''
std_perturb_Lambda_num = np.std(Beigvseries[:,:,0],axis=1)
figE,axE=plt.subplots(figsize=(4,4))
cm='rbg'
axE.plot(gaverageseries,std_perturb_Lambda_num,color=cm[1],linewidth=1.5,label='numerical')
# polyfitting
p=np.polyfit(gaverageseries,std_perturb_Lambda_num,1)
std_perturbLambda_linear_num = np.zeros(ngavg)
std_perturbLambda_linear_num = p[0]*gaverageseries+p[1]
# axE.plot(gaverageseries,std_perturbLambda_linear,color=cm[2],linestyle='--',linewidth=1.5,alpha=0.5,label='1-order polynomial')
axE.plot(gaverageseries,std_perturbLambda_linear_theo,color='darkgreen',linewidth=1.5,label='analytical')
print('std delta_lambda numerical:',p[0])
axE.set_title(r'$\sigma_{\lambda}$')

'''
plt and ax
'''
ylims=[-0.00,0.08]
xlims=[-0.0,1.1]
yticks = np.linspace(0.0,0.08,2)
xticks = np.linspace(0.0,1.0,2)
axE.set_xlim(xlims)
axE.set_xticks(xticks)
axE.set_ylim(ylims)
axE.set_yticks(yticks)
# ax.legend()

'''
right eigenvector
'''
## calculating the statistics of perturbation lambda
# Coding uses only one point
std_perturb_Reig_numSE,std_perturb_Reig_theo=np.zeros(2),np.zeros(2)
std_perturb_Reig_numSE[0],std_perturb_Reig_numSE[1] = np.mean(statesReig[ngavg-1,:,0,1]),np.mean(statesReig[ngavg-1,:,1,1])
std_perturb_Reig_theo[0],std_perturb_Reig_theo[1] = np.sqrt(((xee)**2+(xei)**2)/(2*2*NE)/eigvAm[0]**2),np.sqrt(((xie)**2+(xii)**2)/(2*2*NE)/eigvAm[0]**2)
## theoretical curve for standard deviation of the eigenvalue
std_perturbReig_linear_theo,std_perturbReig_linear_numSE=np.zeros((ngavg,2)),np.zeros((ngavg,2))
std_perturbReig_linear_theo[:,0]=std_perturb_Reig_theo[0]*gaverageseries
std_perturbReig_linear_theo[:,1]=std_perturb_Reig_theo[1]*gaverageseries
std_perturbReig_linear_numSE[:,0]=std_perturb_Reig_numSE[0]*gaverageseries
std_perturbReig_linear_numSE[:,1]=std_perturb_Reig_numSE[1]*gaverageseries

print('std of perturbation right eigenvector, num:',std_perturb_Reig_numSE,'; theo:',std_perturb_Reig_theo)
'''
Variance of the perturbations in eigenvalue lambda
\delta lambda =\Delta lambda = (1) unit change in g
'''
std_perturb_Reig_num,std_perturbReig_linear_num=np.zeros((ngavg,2)),np.zeros((ngavg,2))
std_perturb_Reig_num[:,0],std_perturb_Reig_num[:,1] = np.mean(statesReig[:,:,0,1],axis=1),np.mean(statesReig[:,:,1,1],axis=1)

figR0,axR0=plt.subplots(2,1,figsize=(4,4))
cm='rbg'
# >>>>>>>> numerical solution
axR0[0].scatter(gaverageseries,std_perturb_Reig_num[:,0],color='darkred',label='numerical')
axR0[1].scatter(gaverageseries,std_perturb_Reig_num[:,1],color='darkblue',label='numerical')
axR0[0].scatter(gaverageseries,std_perturbReig_linear_numSE[:,0],color='orange',label='numerical')
axR0[1].scatter(gaverageseries,std_perturbReig_linear_numSE[:,1],color='cyan',label='numerical')
# >>>>>>> Theoretical solutions
axR0[0].plot(gaverageseries,std_perturbReig_linear_theo[:,0],color='red',linewidth=1.5,label='analytical')
axR0[1].plot(gaverageseries,std_perturbReig_linear_theo[:,1],color='blue',linewidth=1.5,label='analytical')
axR0[0].set_title(r'$\sigma_{r}^E$')
axR0[1].set_title(r'$\sigma_{r}^I$')

figR,axR=plt.subplots(2,1,figsize=(4,4))
cm='rbg'
# >>>>>>>> numerical solution
for i in range(ntrial):
    axR[0].scatter(gaverageseries,statesReig[:,i,0,1],s=10,c='darkred',marker='^',label='numerical')
    axR[1].scatter(gaverageseries,statesReig[:,i,1,1],s=10,c='darkblue',marker='^',label='numerical')
# >>>>>>> Theoretical solutions
axR[0].plot(gaverageseries,std_perturbReig_linear_theo[:,0],color='red',linewidth=1.5,label='analytical')
axR[1].plot(gaverageseries,std_perturbReig_linear_theo[:,1],color='blue',linewidth=1.5,label='analytical')
axR[0].set_title(r'$\sigma_{r}^E$')
axR[1].set_title(r'$\sigma_{r}^I$')

ylims=[-0.00,0.015]
xlims=[-0.0,1.1]
yticks = np.linspace(0.0,0.015,2)
xticks = np.linspace(0.0,1.0,2)
for i in range(2):
    axR[i].set_xlim(xlims)
    axR[i].set_xticks([])
    axR[i].set_ylim(ylims)
    axR[i].set_yticks(yticks)

    axR0[i].set_xlim(xlims)
    axR0[i].set_xticks([])
    axR0[i].set_ylim(ylims)
    axR0[i].set_yticks(yticks)
axR[1].set_xticks(xticks)
axR0[1].set_xticks(xticks)

'''
left eigenvector
'''
## calculating the statistics of perturbation lambda
# Coding uses only one point
std_perturb_Leig_numSE,std_perturb_Leig_theo=np.zeros(2),np.zeros(2)
std_perturb_Leig_numSE[0],std_perturb_Leig_numSE[1] = np.mean(statesLeig[ngavg-1,:,0,1]),np.mean(statesLeig[ngavg-1,:,1,1])
std_perturb_Leig_theo[0],std_perturb_Leig_theo[1] = np.sqrt(2*((JE*xee)**2+(JI*xie)**2)/(2*NE)/(JE-JI)**2/eigvAm[0]**2),np.sqrt(2*((JE*xei)**2+(JI*xii)**2)/(2*NE)/(JE-JI)**2/eigvAm[0]**2)
## theoretical curve for standard deviation of the eigenvalue
std_perturbLeig_linear_theo,std_perturbLeig_linear_numSE=np.zeros((ngavg,2)),np.zeros((ngavg,2))
std_perturbLeig_linear_theo[:,0]=std_perturb_Leig_theo[0]*gaverageseries
std_perturbLeig_linear_theo[:,1]=std_perturb_Leig_theo[1]*gaverageseries
std_perturbLeig_linear_numSE[:,0]=std_perturb_Leig_numSE[0]*gaverageseries
std_perturbLeig_linear_numSE[:,1]=std_perturb_Leig_numSE[1]*gaverageseries

print('std of perturbation left eigenvector, num:',std_perturb_Leig_numSE,'; theo:',std_perturb_Leig_theo)
'''
Variance of the perturbations in eigenvalue lambda
\delta lambda =\Delta lambda = (1) unit change in g
'''
std_perturb_Leig_num,std_perturbLeig_linear_num=np.zeros((ngavg,2)),np.zeros((ngavg,2))
std_perturb_Leig_num[:,0],std_perturb_Leig_num[:,1] = np.mean(statesLeig[:,:,0,1],axis=1),np.mean(statesLeig[:,:,1,1],axis=1)

figL0,axL0=plt.subplots(2,1,figsize=(4,4))
cm='rbg'
# >>>>>>>> numerical solution
axL0[0].scatter(gaverageseries,std_perturb_Leig_num[:,0],color='darkred',label='numerical')
axL0[1].scatter(gaverageseries,std_perturb_Leig_num[:,1],color='darkblue',label='numerical')
axL0[0].scatter(gaverageseries,std_perturbLeig_linear_numSE[:,0],color='orange',label='numerical')
axL0[1].scatter(gaverageseries,std_perturbLeig_linear_numSE[:,1],color='cyan',label='numerical')
# >>>>>>> Theoretical solutions
axL0[0].plot(gaverageseries,std_perturbLeig_linear_theo[:,0],color='red',linewidth=1.5,label='analytical')
axL0[1].plot(gaverageseries,std_perturbLeig_linear_theo[:,1],color='blue',linewidth=1.5,label='analytical')
axL0[0].set_title(r'$\sigma_{r}^E$')
axL0[1].set_title(r'$\sigma_{r}^I$')

figL,axL=plt.subplots(2,1,figsize=(4,4))
cm='mg'#'rb'#
# >>>>>>>> numerical solution
for i in range(ntrial):
    axL[0].scatter(gaverageseries,statesLeig[:,i,0,1],s=10,c='darkred',marker='^',label='numerical')
    axL[1].scatter(gaverageseries,statesLeig[:,i,1,1],s=10,c='darkblue',marker='^',label='numerical')
# >>>>>>> Theoretical solutions
axL[0].plot(gaverageseries,std_perturbLeig_linear_theo[:,0],color='red',linewidth=1.5,label='analytical')
axL[1].plot(gaverageseries,std_perturbLeig_linear_theo[:,1],color='blue',linewidth=1.5,label='analytical')
axL[0].set_title(r'$\sigma_{l}^E$')
axL[1].set_title(r'$\sigma_{l}^I$')

ylims=[-0.00,0.025]
xlims=[-0.0,1.1]
yticks = np.linspace(0.0,0.025,2)
xticks = np.linspace(0.0,1.0,2)
for i in range(2):
    axL[i].set_xlim(xlims)
    axL[i].set_xticks([])
    axL[i].set_ylim(ylims)
    axL[i].set_yticks(yticks)

    axL0[i].set_xlim(xlims)
    axL0[i].set_xticks([])
    axL0[i].set_ylim(ylims)
    axL0[i].set_yticks(yticks)
axL[1].set_xticks(xticks)
axL0[1].set_xticks(xticks)




std delta_lambda numerical: 0.018832070565040642
std of perturbation right eigenvector, num: [0.01040604 0.00298035] ; theo: [0.01       0.00282843]
<ipython-input-21-bc4f9400eef4>:57: ComplexWarning: Casting complex values to real discards the imaginary part
  std_perturb_Reig_numSE[0],std_perturb_Reig_numSE[1] = np.mean(statesReig[ngavg-1,:,0,1]),np.mean(statesReig[ngavg-1,:,1,1])
<ipython-input-21-bc4f9400eef4>:58: ComplexWarning: Casting complex values to real discards the imaginary part
  std_perturb_Reig_theo[0],std_perturb_Reig_theo[1] = np.sqrt(((xee)**2+(xei)**2)/(2*2*NE)/eigvAm[0]**2),np.sqrt(((xie)**2+(xii)**2)/(2*2*NE)/eigvAm[0]**2)
<ipython-input-21-bc4f9400eef4>:72: ComplexWarning: Casting complex values to real discards the imaginary part
  std_perturb_Reig_num[:,0],std_perturb_Reig_num[:,1] = np.mean(statesReig[:,:,0,1],axis=1),np.mean(statesReig[:,:,1,1],axis=1)
D:\UseApp\Anaconda3\lib\site-packages\numpy\core\_asarray.py:138: ComplexWarning: Casting complex values to 

## For Case 3 and 4, with symmetric connectivity
* Based on determinant lemma, calculate eigenvalue outlier (spectrum of eigenvalues)
* Based on perturbation theorem, calculate the statistics of eigenvectors (eigenvector perturbations, MEAN)
* Validate the results, in the case with homogeneous random gain and the case with heterogeneous random gain 

In [232]:
'''
WITH SYMMETRY
'''
'''
Cases 3,4 with symmetry
Focus on mean 
Focus on second moment
Focus on derivative on 
'''
# generate mean matrix
nrank=1
Nt=np.array([1000,1000])
NE,NI=Nt[0],Nt[1]
N=NE+NI
Nparams=np.array([NE,NI])
'''
>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
'''
nrank,ntrial,neta,ngavg=1,20,10,10

## for all casesq
xticks = np.linspace(-4.0,3.0,5)
xlims = [-4.1,3.1]
yticks = np.linspace(-1.3,1.3,3)
ylims = [-1.31,1.31]
theta = np.linspace(0, 2 * np.pi, 200)

''' C0 rules '''
Xtest = np.zeros((NE*2,NE*2))
Xtest[:NE,:NE]=1
idxEE,idyEE=np.where(Xtest>0)
lowinEE=np.where(idxEE<idyEE)
upinEE=np.where(idxEE>idyEE)
idxUpEE,idyUpEE,idxDownEE,idyDownEE=idxEE[lowinEE],idyEE[lowinEE],idxEE[upinEE],idyEE[upinEE]
Xrep = np.zeros_like(Xtest)
Xrep[idxUpEE,idyUpEE]=1
idxUpII,idyUpII,idxDownII,idyDownII=idxUpEE+NE,idyUpEE+NE,idxDownEE+NE,idyDownEE+NE

''' 
---------- method 2 ------------------ 
'''
''' ## Three \bar{J} cases '''
JI,JE,a,b=3.5,1.00,-0.6,0.6 # already recording -3.5# 2.0
a,b=0.0,0.0
# JI,JE,a,b=3.5,1.0,3.5,1.0 # already recording -3.5# 2.0
# JI,JE=0,0
JEE,JIE,JEI,JII=JE+a,JE-a,JI-b,JI+b
''' Am -- J(g0=0), xAm(g0=0), yAm(g0=0) '''
Am,Jsv=generate_meanmat_eig(Nparams,JEE,JIE,JEI,JII)
# calculate the original \bar{m} and \bar{n}
eigvAm,eigveclAm,eigvecrAm=scpla.eig(Am,left=True,right=True)
meigvecAm=eigvecrAm.copy()
neigvecAm=eigveclAm.copy()
for i in range(nrank):
    normalizecoe=(np.sum(neigvecAm[:,i]*meigvecAm[:,i]))
    neigvecAm[:,i]*=(eigvAm[i]/normalizecoe)
for i in range(1,NE*2):
    normalizecoe=(np.sum(neigvecAm[:,i]*meigvecAm[:,i]))
    neigvecAm[:,i]/=normalizecoe
''' for reference '''
xAm,yAm=np.reshape(meigvecAm[:,0],(NE*2,1)),np.reshape(neigvecAm[:,0].copy()/eigvAm[0],(NE*2,1))

'''
##>>>>>>>>>>> g0!=0, one example >>>>>>>>
'''
gaverage=0.2
# X0=np.random.randn(NE*2,NE*2)*gaverage/np.sqrt(2*NE) # /np.sqrt(2) no later
X0=np.zeros((NE*2,NE*2))
X0[:NE,:NE]=np.reshape(np.random.randn(NE*NE),(NE,NE))*gaverage/np.sqrt(2*NE)
X0[NE:,NE:]=np.reshape(np.random.randn(NE*NE),(NE,NE))*gaverage/np.sqrt(2*NE)
X0[NE:,:NE]=np.reshape(np.random.randn(NE*NE),(NE,NE))*gaverage/np.sqrt(2*NE)
X0[:NE,NE:]=np.reshape(np.random.randn(NE*NE),(NE,NE))*gaverage/np.sqrt(2*NE)
Xsym=X0.copy()
XsymT=Xsym.copy().T
J0=Am.copy()+X0.copy()
## >>>>>> calculate the original \bar{m} and \bar{n} 
eigvJ0,eigrvecJ0=la.eig(J0)
inveigrvecJ0=la.inv(eigrvecJ0)
meig0 = np.squeeze(eigrvecJ0[:,:].copy())
neig0 = np.squeeze(inveigrvecJ0[:,:].copy()) # inverse
neig0=neig0.copy().T
## test neig0*meig0=1
print('test normal:',np.sum(neig0[:,0]*meig0[:,0]))
for i in range(nrank):
    neig0[:,i]*=eigvJ0[i]
for j in range(1,N):
    neig0[:,j]*=eigvJ0[j]
## >>>>> eigendecomposition 
leig0, reig0 = np.zeros((N,N)),np.zeros((N,N))
''' The first '''
reig0=meig0.copy()
normval=np.sum(reig0*neig0.copy(),axis=0)
normval=np.repeat(np.reshape(normval,(1,N)),N,axis=0)
leig0=neig0.copy()/normval.copy()
''' calculate s0'''
Z = np.zeros((NE*2-1,NE*2-1))
for i in range(NE*2-1):
    b1=eigvJ0[i+1]
    Z[i,i]=1/(b1-eigvJ0[0])
S0=reig0[:,1:]@Z@leig0[:,1:].T
print('test normal 2:',np.sum(leig0[:,0]*reig0[:,0]))
print('test normal 3:',np.sum(meig0[:,0]*neig0[:,0]))
print('eigenvalues:',eigvJ0[:nrank])
'''## for reference '''
x0,y0=np.reshape(reig0[:,0],(NE*2,1)),np.reshape(leig0[:,0].copy(),(NE*2,1))
''' ## normalization 0 (reference) absolute reference is xAm, yAm '''
tildex,tildey=x0[:,0].copy(),y0[:,0].copy()
## make sure the E in x is positive, if negative then all x and y should change sign
if (np.mean(tildex[:NE])*xAm[0,0])<0:
    tildex*=(-1)
    tildey*=(-1)
x0,y0=np.reshape(tildex,(NE*2,1)),np.reshape(tildey,(NE*2,1))
## >>>> renormalization, mu_l and mu_r are unchanged
x0=np.sqrt(np.squeeze(tildey.T@xAm)/np.squeeze(yAm.T@tildex))*tildex
y0=np.sqrt(np.squeeze(tildex.T@yAm)/np.squeeze(xAm.T@tildey))*tildey
x0,y0=np.reshape(x0,(NE*2,1)),np.reshape(y0,(NE*2,1))
'''
##>>>>>>>>>>> end >>>>>>>>
'''

'''
Network Setting for Iterating Following
'''
signeta=np.ones(3)
# signeta[1]*=(-1)
## heterogeneous random gain
''' >>>>>>>>>>>>>> Case 1 and Case 2 >>>>>>>>>>>>>>>>>>>>>>> '''
xee,xei,xie,xii=1.0,1.0,1.0,1.0 ## HOMOgeneous random gain g_kl
# xee,xei,xie,xii=1.0,0.5,0.2,0.8 ## HETEROgeneous random gain
# coeffeta=np.array([1.0,0.2,0.8]) ## HETERogeneous symmetry D_{\rho}
# coeffeta=np.array([1.0,0.8,1.0])
coeffeta=np.array([1.0,1.0,1.0])
'''
Recording Variables
'''
# original data
eigvalseries,mvecseries,nvecseries=np.zeros((ngavg,ntrial,nrank),dtype=complex),np.zeros((ngavg,ntrial,NE*2,nrank),dtype=complex),np.zeros((ngavg,ntrial,NE*2,nrank),dtype=complex)
Reigvecseries,Leigvecseries=np.zeros((ngavg,ntrial,NE*2,N),dtype=complex),np.zeros((ngavg,ntrial,NE*2,N),dtype=complex)
Beigvseries=np.zeros((ngavg,ntrial,N),dtype=complex)
''' All recorded connectivity statistics '''
statesReig,statesLeig=np.zeros((ngavg,ntrial,3,2),dtype=complex),np.zeros((ngavg,ntrial,3,2),dtype=complex) #E,I,Total/mean,std# 
coveigvecseries=np.zeros((ngavg,ntrial,3),dtype=complex) # E, I and total
''' All random matrices for each trials '''
## Not X total (symmetry), nut Y independent
gYbasexee=np.zeros((ntrial,NE*2,NE*2))

''' Iterative Processing '''
etaseries=np.linspace(0.0,1.0,neta)
for iktrial in range(ntrial):
    '''
    ##>>>>>>>>>>> g0!=0, >>>>>>>>
    '''
    X0=np.random.randn(NE*2,NE*2)*gaverage/np.sqrt(2*NE) # /np.sqrt(2) no later
    Xsym=X0.copy()
    XsymT=Xsym.copy().T

    J0=Am.copy()+X0.copy()
    ''' # calculate the original \bar{m} and \bar{n} '''
    eigvJ0,eigrvecJ0=la.eig(J0)
    inveigrvecJ0=la.inv(eigrvecJ0)
    meig0 = np.squeeze(eigrvecJ0[:,:].copy())
    neig0 = np.squeeze(inveigrvecJ0[:,:].copy()) # inverse
    neig0=neig0.copy().T
    print('test normal:',np.sum(neig0[:,0]*meig0[:,0]))
    for i in range(nrank):
        neig0[:,i]*=eigvJ0[i]
    for j in range(1,N):
        neig0[:,j]*=eigvJ0[j]

    leig0, reig0 = np.zeros((N,N)),np.zeros((N,N))
    ''' The first '''
    reig0=meig0.copy()
    normval=np.sum(reig0*neig0.copy(),axis=0)
    normval=np.repeat(np.reshape(normval,(1,N)),N,axis=0)
    leig0=neig0.copy()/normval.copy()
    ''' calculate s0'''
    Z = np.zeros((NE*2-1,NE*2-1))
    for i in range(NE*2-1):
        b1=eigvJ0[i+1]
        Z[i,i]=1/(b1-eigvJ0[0])
    S0=reig0[:,1:]@Z@leig0[:,1:].T
    print('test normal 2:',np.sum(leig0[:,0]*reig0[:,0]))
    print('test normal 3:',np.sum(meig0[:,0]*neig0[:,0]))
    print('eigenvalues:',eigvJ0[:nrank])
    '''## for reference '''
    x0,y0=np.reshape(reig0[:,0],(NE*2,1)),np.reshape(leig0[:,0].copy(),(NE*2,1))
    ''' ## normalization 0 (reference) absolute reference is xAm, yAm '''
    tildex,tildey=np.reshape(x0[:,0].copy(),(NE*2,1)),np.reshape(y0[:,0].copy(),(NE*2,1))
    ## make sure the E in x is positive, if negative then all x and y should change sign
    if (np.mean(tildex[:NE])*xAm[0,0])<0:
        tildex*=(-1)
        tildey*=(-1)
    x0,y0=np.reshape(tildex,(NE*2,1)),np.reshape(tildey,(NE*2,1))
    x0=np.sqrt(np.squeeze(tildey.T@xAm)/np.squeeze(yAm.T@tildex))*tildex
    y0=np.sqrt(np.squeeze(tildex.T@yAm)/np.squeeze(xAm.T@tildey))*tildey
    x0,y0=np.reshape(x0,(NE*2,1)),np.reshape(y0,(NE*2,1))
    for idxeta,eta in enumerate(etaseries):
        etaset=eta*np.ones(3)#eta*np.ones(3)
        for icoeff in range(3):
          etaset[icoeff]*=coeffeta[icoeff]
        # etaset[0]*=0.1
        Xinit = Xsym.copy()

        ''' heterogeneous ETA method 1 '''
        ## EE ##
        asqr=(1-np.sqrt(1-etaset[0]**2))/2.0 ## when eta = 0, asqr = 0, aamp = 0, XT-0, X-1
        aamp=np.sqrt(asqr)
        Xinit[:NE,:NE]=signeta[0]*aamp*XsymT[:NE,:NE].copy()+np.sqrt(1-aamp**2)*Xsym[:NE,:NE].copy()
        ## EI IE##
        asqr=(1-np.sqrt(1-etaset[1]**2))/2.0
        aamp=np.sqrt(asqr)
        Xinit[NE:,:NE]=signeta[1]*aamp*XsymT[NE:,:NE].copy()+np.sqrt(1-aamp**2)*Xsym[NE:,:NE].copy()
        Xinit[:NE,NE:]=signeta[1]*aamp*XsymT[:NE,NE:].copy()+np.sqrt(1-aamp**2)*Xsym[:NE,NE:].copy()
        ## II ##
        asqr=(1-np.sqrt(1-etaset[2]**2))/2.0
        aamp=np.sqrt(asqr)
        Xinit[NE:,NE:]=signeta[2]*aamp*XsymT[NE:,NE:].copy()+np.sqrt(1-aamp**2)*Xsym[NE:,NE:].copy()
        ''' THEORETICAL FULL ETA '''
        symtheofull=gaverage**2*(etaset[0]*signeta[0]+etaset[2]*signeta[2]+2*etaset[1]*signeta[1])/4.0#+(a-b)**2/(NE*2)
        X=Xinit.copy()
        # incase heterogeneous
        X[:NE,:NE]*=(xee)
        X[:NE,NE:]*=(xei)
        X[NE:,:NE]*=(xie)
        X[NE:,NE:]*=(xii)

        # overall
        J = X.copy()+Am.copy()
        JT = J.copy().T
        symnumfull=np.sum(np.sum(J*JT))/(4*NE**2)-np.mean(np.mean(J.copy()))**2
        symnumfull*=2*NE
        print('compare degree of symmetry:',symnumfull,'--num; ',etaset*signeta,'--setting')
        # properties
        eigvJ,eigrvecJ=la.eig(J)
        inveigrvecJ=la.inv(eigrvecJ)
        meig = np.squeeze(eigrvecJ[:,:].copy())
        # meig = np.reshape(meig,(NE*2,nrank))
        neig = np.squeeze(inveigrvecJ[:,:].copy()) # inverse
        neig=neig.copy().T
        print('test normal 4:',np.sum(neig[:,0]*meig[:,0]))
        for i in range(nrank):
            neig[:,i]*=eigvJ[i]
        for j in range(1,N):
            neig[:,j]*=eigvJ[j]
        ## >>> for low rank framework m and n
        eigvalseries[idxeta,iktrial,0],mvecseries[idxeta,iktrial,:,0],nvecseries[idxeta,iktrial,:,0]=eigvJ[0],meig[:,0].copy(),neig[:,0].copy()
        
        ## >>> for eigendecomposition 
        Leig, Reig = np.zeros((N,N)),np.zeros((N,N))
        ''' The first '''
        Reig=meig.copy()
        normval=np.sum(Reig*neig.copy(),axis=0)
        normval=np.repeat(np.reshape(normval,(1,N)),N,axis=0)
        Leig=neig.copy()/normval.copy()
        print('test normal 5:',np.sum(Leig[:,0]*Reig[:,0]))
        '''
        normalization 0 (reference)
        '''
        tildex,tildey=Reig[:,0].copy(),Leig[:,0].copy()
        if (np.mean(tildex[:NE])*xAm[0,0])<0:
            tildex*=(-1)
            tildey*=(-1)
        xnorm0=np.sqrt(np.squeeze(tildey.T@x0)/np.squeeze(y0.T@tildex))*tildex
        ynorm0=np.sqrt(np.squeeze(tildex.T@y0)/np.squeeze(x0.T@tildey))*tildey

        Reigvecseries[idxeta,iktrial,:,:],Leigvecseries[idxeta,iktrial,:,:]=Reig.copy(),Leig.copy()
        Beigvseries[idxeta,iktrial,:]=eigvJ.copy()
        Reigvecseries[idxeta,iktrial,:,0],Leigvecseries[idxeta,iktrial,:,0]=xnorm0[:].copy(),ynorm0[:].copy()

        '''  RECORDING STATES '''
        statesReig[idxeta,iktrial,0,0],statesReig[idxeta,iktrial,1,0]=np.mean(Reigvecseries[idxeta,iktrial,:NE,0]),np.mean(Reigvecseries[idxeta,iktrial,NE:,0])
        statesReig[idxeta,iktrial,2,0]=np.mean(Reigvecseries[idxeta,iktrial,:,0])
        statesReig[idxeta,iktrial,0,1],statesReig[idxeta,iktrial,1,1]=np.std(Reigvecseries[idxeta,iktrial,:NE,0]),np.std(Reigvecseries[idxeta,iktrial,NE:,0])
        statesReig[idxeta,iktrial,2,1]=np.std(Reigvecseries[idxeta,iktrial,:,0])
       
        statesLeig[idxeta,iktrial,0,0],statesLeig[idxeta,iktrial,1,0]=np.mean(Leigvecseries[idxeta,iktrial,:NE,0]),np.mean(Leigvecseries[idxeta,iktrial,NE:,0])
        statesLeig[idxeta,iktrial,2,0]=np.mean(Leigvecseries[idxeta,iktrial,:,0])
        statesLeig[idxeta,iktrial,0,1],statesLeig[idxeta,iktrial,1,1]=np.std(Leigvecseries[idxeta,iktrial,:NE,0]),np.std(Leigvecseries[idxeta,iktrial,NE:,0])
        statesLeig[idxeta,iktrial,2,1]=np.std(Leigvecseries[idxeta,iktrial,:,0])

        ''' covariance '''
        '''
        Note, we should change the normalization for the first item to the (Refer) normalization 
        tildeReig,tildeLeig
        normalization 0
        '''
        ## normalization 0 reference
        tildeReig,tildeLeig=np.squeeze(Reigvecseries[idxeta,iktrial,:,:]),np.squeeze(Leigvecseries[idxeta,iktrial,:,:])
        reig,leig=np.reshape(tildeReig[:,0].copy(),(NE*2,1)),np.reshape(tildeLeig[:,0].copy(),(NE*2,1))
        ## mean at eact point
        rEceneig,rIceneig = np.mean(reig[:NE,0]),np.mean(reig[NE:,0])
        lEceneig,lIceneig = np.mean(leig[:NE,0]),np.mean(leig[NE:,0])
        ## random components
        rrandeig,lrandeig=reig.copy(),leig.copy()
        rrandeig[:NE,0],rrandeig[NE:,0]=rrandeig[:NE,0]-rEceneig,rrandeig[NE:,0]-rIceneig
        lrandeig[:NE,0],lrandeig[NE:,0]=lrandeig[:NE,0]-lEceneig,lrandeig[NE:,0]-lIceneig

        coveigvecseries[idxeta,iktrial,0]=np.reshape(rrandeig[:NE,0],(NE,1)).T@np.reshape(lrandeig[:NE,0],(NE,1))
        coveigvecseries[idxeta,iktrial,1]=np.reshape(rrandeig[NE:,0],(NE,1)).T@np.reshape(lrandeig[NE:,0],(NE,1))
        coveigvecseries[idxeta,iktrial,2]=coveigvecseries[idxeta,iktrial,0]+coveigvecseries[idxeta,iktrial,1]
        ''' record the random derivative '''
        if idxeta == 0:
            print('recording!!!')
            ## log: there is a bug, X is the total random xconn, including symmetry
            gYbasexee[iktrial,:,:]=Xsym.copy()


compare degree of symmetry: 0.017837081038737015 --num;  [0.44444444 0.44444444 0.44444444] --setting
test normal 4: (1.0000000000000133+2.964949361374549e-14j)
test normal 5: (1.0000000000000004+0j)
compare degree of symmetry: 0.022281324830451265 --num;  [0.55555556 0.55555556 0.55555556] --setting
test normal 4: (1.0000000000000024-2.399040228193322e-14j)
test normal 5: (0.9999999999999994+7.257520328033309e-29j)
compare degree of symmetry: 0.026725547509261805 --num;  [0.66666667 0.66666667 0.66666667] --setting
test normal 4: (0.9999999999999616-1.3144621742178812e-14j)
test normal 5: (1.0000000000000018+3.470987982972452e-29j)
compare degree of symmetry: 0.031169741262574394 --num;  [0.77777778 0.77777778 0.77777778] --setting
test normal 4: (1.0000000000000144+2.133438695650051e-14j)
test normal 5: (1-7.573064690121713e-29j)
compare degree of symmetry: 0.035613884860624494 --num;  [0.88888889 0.88888889 0.88888889] --setting
test normal 4: (1.0000000000000009+9.495642760660468e-

In [234]:
# saving data
now = time.strftime("%Y-%m-%d-%H_%M_%S",time.localtime(time.time()))
Jparameters = np.zeros(4+1)
Jparameters[0],Jparameters[1],Jparameters[2],Jparameters[3]=JEE,JEI,JIE,JII
Jparameters[4]=NE
Randgain=np.zeros(4)
Randgain[0],Randgain[1],Randgain[2],Randgain[3]=xee,xei,xie,xii

np.savez(now+'_perturbationHomoeta_data',x0=x0,y0=y0,xAm=xAm,yAm=yAm,Jparameters=Jparameters,Randgain=Randgain,gaverage=gaverage,etaseries=etaseries,etaset=etaset,signeta=signeta,eigvalseries=eigvalseries,Reigvecseries=Reigvecseries[:,:,:,0],Leigvecseries=Leigvecseries[:,:,:,0],Beigvseries=Beigvseries[:,:,0],meigvecAm=meigvecAm[:,0],neigvecAm=neigvecAm[:,0],eigvAm=eigvAm,statesReig=statesReig,statesLeig=statesLeig,coveigvecseries=coveigvecseries,coveigvecAmseries=coveigvecAmseries,gYbasexee=gYbasexee,coeffeta=coeffeta)


In [210]:
'''
with symmetry, 
eigvJ0
'''
idxgavg,idxtrial=0,8 # 6,3,0
# figtspt,axtspt=plt.subplots(figsize=(6,3))
cm='bgc'
idxc=2
axtspt.scatter(np.real(Beigvseries[idxgavg,idxtrial,1:]),np.imag(Beigvseries[idxgavg,idxtrial,1:]),s=5,c=cm[idxc],alpha=0.25) # >>>>>>>>>>>>>>
axtspt.scatter(np.real(Beigvseries[idxgavg,:,0]),np.imag(Beigvseries[idxgavg,:,0])-0.2*idxc,s=20,c=cm[idxc],marker='^',alpha=0.5) # >>>>>>>>>>>>
axtspt.set_aspect('equal')
axtspt.plot(np.ones(2)*eigvAm[0],np.array([-1.0,1.0]),linestyle='--',linewidth=1.5,color='grey')
axtspt.spines['right'].set_color('none')
axtspt.spines['top'].set_color('none')
#设置x坐标刻度数字或名称的位置
axtspt.xaxis.set_ticks_position('bottom')
#设置边框位置
axtspt.spines['bottom'].set_position(('data', 0))

aee,aei,aie,aii=xee,xei,xie,xii
print(xee,xei,xie,xii)
idxtrial=10
eta=0
xticks = np.linspace(-1.3,1.3,2)
xlims = [-1.3,1.3]
yticks = np.linspace(-1.3,1.3,2)
ylims = [-1.31,1.31]
theta = np.linspace(0, 2 * np.pi, 200)
# first do not multiply at
ahomo=gaverage
xee_,xei_,xie_,xii_=ahomo*aee/np.sqrt(2),ahomo*aei/np.sqrt(2),ahomo*aie/np.sqrt(2),ahomo*aii/np.sqrt(2)
gmat = np.array([[xee_**2,xei_**2],[xie_**2,xii_**2]])
gaverage_=0
for i in range(2):
    for j in range(2):
        gaverage_+=gmat[i,j]/2 # ntype=2
gaverage_=np.sqrt(gaverage_)
eigvgm,eigvecgm=la.eig(gmat) 
r_g2=np.max(eigvgm)
r_g = np.sqrt(r_g2)
print(r_g)
eta=etaseries[idxgavg]
longaxis,shortaxis=(1+eta)*r_g ,(1-eta)*r_g 
xr = longaxis*np.cos(theta)
yr = shortaxis*np.sin(theta)
axtspt.plot(xr, yr, color="gray", linewidth=2.5,linestyle='--',label=r'ellipse') # >>>>>
axtspt.plot(longaxis*np.ones(2), np.array([-1.0,1.0]), color="gray", linewidth=2.5,linestyle='--') # >>>>>
axtspt.set_title(r'$\eta=$'+np.str(eta))
xticks = np.linspace(-3.5,2.5,2)
xlims = [-3.5,2.5]
yticks = np.linspace(-1.0,1.0,2)
ylims = [-1.1,1.1]
axtspt.set_xlim(xlims)
axtspt.set_ylim(ylims)
axtspt.set_xticks(xticks)
axtspt.set_yticks(yticks)
axtspt.set_aspect('equal')

1.0 1.0 1.0 1.0
0.8
/Applications/Anaconda/anaconda3/lib/python3.8/site-packages/numpy/core/_asarray.py:85: ComplexWarning: Casting complex values to real discards the imaginary part
  return array(a, dtype, copy=False, order=order)


In [202]:
axtspt.plot(xr, yr, color="pink", linewidth=2.5,linestyle='--',label=r'ellipse') # >>>>>
axtspt.plot(longaxis*np.ones(2), np.array([-1.0,1.0]), color="pink", linewidth=2.5,linestyle='--') # >>>>>
axtspt.set_title(r'$\eta=$'+np.str(eta))
xticks = np.linspace(-3.5,2.5,2)
xlims = [-3.5,2.5]
yticks = np.linspace(-1.0,1.0,2)
ylims = [-1.1,1.1]
axtspt.set_xlim(xlims)
axtspt.set_ylim(ylims)
axtspt.set_xticks(xticks)
axtspt.set_yticks(yticks)
axtspt.set_aspect('equal')

## individual perturbations of left and right eigenvectors
* calculate the predicted perturbations (theo)
* comparing with numerical perturbations (num)

## population statistics 
* first determint lemma for eigenvalue outlier -- change

In [222]:
'''
determinant lemma, odd terms 
even order terms should be maitained
'''
xticks = np.linspace(-2.91,-2.5,2)
xlims = [-2.91,-2.5]
yticks = np.linspace(-2.91,-2.5,2)
ylims = [-2.91,-2.5]

coeff=coeffeta.copy()
nAm=np.reshape(yAm*eigvAm[0],(NE*2,1))
mAm=np.reshape(xAm.copy(),(NE*2,1))
nAm,mAm=np.real(nAm),np.real(mAm)
y0E,y0I=np.mean(y0[:NE,0]),np.mean(y0[NE:,0])
x0E,x0I=np.mean(x0[:NE,0]),np.mean(x0[NE:,0])
lambda_theo2 = np.zeros(neta)
lambda_theo4 = np.zeros(neta)
lambda_num = np.mean(np.squeeze(Beigvseries[:,:,0]),axis=1)
cutoff = 2

for idxeta,eta in enumerate(etaseries):
    etaE,etaB,etaI=eta*coeff[0],eta*coeff[1],eta*coeff[2]
    '''
    Even-order terms have basic element XX.T(X.TX)
    '''
    bXXT=np.eye(NE*2)
    bXXT[:NE,:NE]*=(etaE*signeta[0]+etaB*signeta[1])/4
    bXXT[NE:,NE:]*=(etaI*signeta[2]+etaB*signeta[1])/4
    # cut off = 2
    C=comb(2,1)
    print(C)
    B2=C*(nAm.T@la.matrix_power(bXXT,1)@mAm)*gaverage**2
    coeff2 = [1,-eigvJ0[0],0,-B2]
    C=comb(4,2)
    B4=C*(nAm.T@la.matrix_power(bXXT,2)@mAm)*gaverage**4
    # B4=(nAm.T@(Y_t@YT_t@Y_t@YT_t)@mAm)*fory**2*foryt**2*6
    coeff4 = [1,-eigvJ0[0],0,-B2,0,-B4]
    # B2=(bcoeffE0*y0E*x0E*NE)+(bcoeffI0*y0I*x0I*NE)
    # coeff2 = [1,-eigvJ0[0],0,-B2*eigvJ0[0]]
    # B4=(bcoeffE0**2*y0E*x0E*NE)+(bcoeffI0**2*y0I*x0I*NE)
    # coeff4 = [1,-eigvJ0[0],0,-B2*eigvJ0[0],0,-B4*eigvJ0[0]]#(eta*eigvAm[0]*gaverage])
    lambda_test2=np.roots(coeff2)
    lambda_theo2[idxeta]=lambda_test2[0]
    lambda_test4=np.roots(coeff4)
    lambda_theo4[idxeta]=lambda_test4[0]
    print(lambda_test2[0],lambda_test4[0])
fig,ax=plt.subplots(figsize=(5,5))  
ax.plot(lambda_num,lambda_theo2,'b',label='2 cut off')
ax.plot(lambda_num,lambda_theo4,'r',label='4 cut off')
# ax.plot(lambda_num,lambda_theo,'b')
ax.plot(lambda_num,lambda_num,'k',label='real')
# ax.set_xlim(xlims)
# ax.set_ylim(ylims)
# ax.set_xticks(xticks)
# ax.set_yticks(yticks)
# ax.set_aspect('equal')
ax.legend()

fig,ax=plt.subplots(figsize=(5,3))  
ax.plot(etaseries,lambda_num,'b',label='2 cut off')
ax.set_aspect('equal')
# ax.scatter(np.real(Beigvseries[neta-5,8,1:]),np.imag(Beigvseries[neta-5,8,1:]))
# ax.set_aspect('equal')


2.0
-2.4526836103346934 -2.4526836103346934
2.0
(-2.4815525297079066+0j) (-2.482040827245587+0j)
2.0
(-2.5091579880528077+0j) (-2.5109839364156423+0j)
2.0
(-2.5356352843721313+0j) (-2.5394874123758466+0j)
2.0
(-2.5610975722304836+0j) (-2.5675361928461964+0j)
2.0
(-2.585640596411454+0j) (-2.595123090657107+0j)
2.0
(-2.6093462047687037+0j) (-2.6222467871533164+0j)
2.0
(-2.632285000948217+0j) (-2.648910287657211+0j)
2.0
(-2.654518381336418+0j) (-2.6751197388731383+0j)
2.0
/Applications/Anaconda/anaconda3/lib/python3.8/site-packages/numpy/lib/polynomial.py:238: ComplexWarning: Casting complex values to real discards the imaginary part
  p = p.astype(float)
<ipython-input-222-6b0c0320941f>:43: ComplexWarning: Casting complex values to real discards the imaginary part
  lambda_theo2[idxeta]=lambda_test2[0]
<ipython-input-222-6b0c0320941f>:45: ComplexWarning: Casting complex values to real discards the imaginary part
  lambda_theo4[idxeta]=lambda_test4[0]
(-2.6761001221586493+0j) (-2.70088352

In [221]:
xticks = np.linspace(-2.91,-2.5,2)
xlims = [-2.91,-2.5]
yticks = np.linspace(-2.91,-2.5,2)
ylims = [-2.91,-2.5]

coeff=coeffeta.copy()
y0E,y0I=np.mean(y0[:NE,0]),np.mean(y0[NE:,0])
x0E,x0I=np.mean(x0[:NE,0]),np.mean(x0[NE:,0])
lambda_theo2 = np.zeros(neta)
lambda_theo4 = np.zeros(neta)
lambda_num = np.mean(np.squeeze(Beigvseries[:,:,0]),axis=1)
cutoff = 2
for idxeta,eta in enumerate(etaseries):
    etaf,etam,etal=eta*coeff[0],coeff[1]*eta,eta*coeff[2]
    bcoeffE0,bcoeffI0=(signeta[0]*etaf+signeta[1]*etam)*1/2.0,(signeta[2]*etal+signeta[1]*etam)*1/2.0
    print(bcoeffI0,bcoeffE0)
    B2=(bcoeffE0*JE-bcoeffI0*JI)*gaverage**2
    coeff2 = [1,-eigvJ0[0],0,-B2]
    B4=(bcoeffE0**2*JE-bcoeffI0**2*JI)*gaverage**4
    coeff4 = [1,-eigvJ0[0],0,-B2,0,-B4]
    # B2=(bcoeffE0*y0E*x0E*NE)+(bcoeffI0*y0I*x0I*NE)
    # coeff2 = [1,-eigvJ0[0],0,-B2*eigvJ0[0]]
    # B4=(bcoeffE0**2*y0E*x0E*NE)+(bcoeffI0**2*y0I*x0I*NE)
    # coeff4 = [1,-eigvJ0[0],0,-B2*eigvJ0[0],0,-B4*eigvJ0[0]]#(eta*eigvAm[0]*gaverage])
    lambda_test2=np.roots(coeff2)
    lambda_theo2[idxeta]=lambda_test2[0]
    lambda_test4=np.roots(coeff4)
    lambda_theo4[idxeta]=lambda_test4[0]
    print(lambda_test2[0],lambda_test4[0])
fig,ax=plt.subplots(figsize=(5,5))  
ax.plot(lambda_num,lambda_theo2,'b',label='2 cut off')
ax.plot(lambda_num,lambda_theo4,'r',label='4 cut off')
# ax.plot(lambda_num,lambda_theo,'b')
ax.plot(lambda_num,lambda_num,'k',label='real')
# ax.set_xlim(xlims)
# ax.set_ylim(ylims)
# ax.set_xticks(xticks)
# ax.set_yticks(yticks)
# ax.set_aspect('equal')
ax.legend()

fig,ax=plt.subplots(figsize=(5,3))  
ax.plot(etaseries,lambda_num,'b',label='2 cut off')
ax.set_aspect('equal')
# ax.scatter(np.real(Beigvseries[neta-5,8,1:]),np.imag(Beigvseries[neta-5,8,1:]))
# ax.set_aspect('equal')


0.0 0.0
(-2.4526836103346934+0j) (-2.4526836103346934+0j)
0.1111111111111111 0.1111111111111111
(-2.4815525297079057-5.551115123125783e-17j) (-2.481878146031667-1.1102230246251565e-16j)
0.2222222222222222 0.2222222222222222
(-2.509157988052805-2.7755575615628914e-16j) (-2.5103764466620673-5.551115123125783e-17j)
0.3333333333333333 0.3333333333333333
(-2.5356352843721313-1.1102230246251565e-16j) (-2.5382084305962955-7.771561172376096e-16j)
0.4444444444444444 0.4444444444444444
(-2.561097572230488+0j) (-2.565403857445122+6.661338147750939e-16j)
0.5555555555555556 0.5555555555555556
(-2.585640596411454+1.1102230246251565e-16j) (-2.5919917948847777-2.220446049250313e-16j)
0.6666666666666666 0.6666666666666666
(-2.6093462047686957-4.440892098500626e-16j) (-2.6180002878036914+0j)
0.7777777777777777 0.7777777777777777
(-2.6322850009482233+3.3306690738754696e-16j) (-2.6434561658680353-3.3306690738754696e-16j)
0.8888888888888888 0.8888888888888888
(-2.6545183813364197+1.1102230246251565e-16j) (

* perturbation theorem for variance of eigenvalue
* perturbation theorem for variance of left and right eigenvectors

In [115]:
'''
variance of r and variance of l <r.Tr>, <l.T,l>
overlap between r and l <l.Tr>
'''
xticks = np.linspace(-0.01,0.1,2)
xlims = [-0.01,0.1]
yticks = np.linspace(-0.01,0.1,2)
ylims = [-0.01,0.1]


itrial=3
gY=np.squeeze(gYbasexee[itrial,:,:])


X0=gY.copy()
J0=Am.copy()+X0.copy()
''' # calculate the original \bar{m} and \bar{n} '''
eigvJ0,eigrvecJ0=la.eig(J0)
inveigrvecJ0=la.inv(eigrvecJ0)
meig0 = np.squeeze(eigrvecJ0[:,:].copy())
neig0 = np.squeeze(inveigrvecJ0[:,:].copy()) # inverse
neig0=neig0.copy().T
print('test normal:',np.sum(neig0[:,0]*meig0[:,0]))
for i in range(nrank):
    neig0[:,i]*=eigvJ0[i]
for j in range(1,N):
    neig0[:,j]*=eigvJ0[j]

leig0, reig0 = np.zeros((N,N)),np.zeros((N,N))
''' The first '''
reig0=meig0.copy()
normval=np.sum(reig0*neig0.copy(),axis=0)
normval=np.repeat(np.reshape(normval,(1,N)),N,axis=0)
leig0=neig0.copy()/normval.copy()
''' calculate s0'''
Z = np.zeros((NE*2-1,NE*2-1))
for i in range(NE*2-1):
    b1=eigvJ0[i+1]
    Z[i,i]=1/(b1-eigvJ0[0])
S0=reig0[:,1:]@Z@leig0[:,1:].T
print('test normal 2:',np.sum(leig0[:,0]*reig0[:,0]))
print('test normal 3:',np.sum(meig0[:,0]*neig0[:,0]))
print('eigenvalues:',eigvJ0[:nrank])
'''## for reference '''
x0,y0=np.reshape(reig0[:,0],(NE*2,1)),np.reshape(leig0[:,0].copy(),(NE*2,1))
''' ## normalization 0 (reference) absolute reference is xAm, yAm '''
tildex,tildey=np.reshape(x0[:,0].copy(),(NE*2,1)),np.reshape(y0[:,0].copy(),(NE*2,1))
## make sure the E in x is positive, if negative then all x and y should change sign
if (np.mean(tildex[:NE])*xAm[0,0])<0:
    tildex*=(-1)
    tildey*=(-1)
x0,y0=np.reshape(tildex,(NE*2,1)),np.reshape(tildey,(NE*2,1))
x0=np.sqrt(np.squeeze(tildey.T@xAm)/np.squeeze(yAm.T@tildex))*tildex
y0=np.sqrt(np.squeeze(tildex.T@yAm)/np.squeeze(xAm.T@tildey))*tildey
x0,y0=np.reshape(x0,(NE*2,1)),np.reshape(y0,(NE*2,1))

'''
rho_prime=eta/(2*sqrt(1-eta**2))
rho'/2/sqrt(rho) to 2nd=(1+np.sqrt(1-etaseries**2))/8/(1-etaseries**2)
rho'/2/sqrt(1-rho) to 2nd=(1-np.sqrt(1-etaseries**2))/8/(1-etaseries**2)
'''
heta=etaseries[1]-etaseries[0]
rhoseries=(1-np.sqrt(1-etaseries**2))/2.0 ## when eta = 0, asqr = 0, aamp = 0, XT-0, X-1
sqrt_1meta2= np.sqrt(1-etaseries**2)
coeffsqrtY,coeffsqrtYT = (1-sqrt_1meta2)/sqrt_1meta2**2/8,(1+sqrt_1meta2)/sqrt_1meta2**2/8
overlaptt,overlapns = np.zeros(neta,dtype=complex),np.zeros(neta,dtype=complex)

for i in range(1,neta):
    Jprimemat=np.sqrt(coeffsqrtYT[i])*gY.copy().T+np.sqrt(coeffsqrtY[i])*gY
    overlaptt[i]=np.squeeze(y0.T@Jprimemat@S0@S0@Jprimemat@x0)
    overlapns[i]=np.squeeze(y0.T@Jprimemat@Jprimemat@x0)/eigvAm[0]**2
overlap = gaverage**2*etaseries/4/(1-etaseries**2)/eigvAm[0]**2
eigenvaluechange = Beigvseries[:,itrial,0]-Beigvseries[0,itrial,0]
stdvaluechange = (statesLeig[:,itrial,2,1]-statesLeig[0,itrial,2,1])**2
stdvaluechange+= (statesReig[:,itrial,2,1]-statesReig[0,itrial,2,1])**2

fig,ax=plt.subplots(figsize=(5,5))  
ax.plot(overlap,coveigvecseries[:,itrial,1],'b',label='2 cut off')
ax.plot(overlapns,coveigvecseries[:,itrial,1],'c',label='2 cut off')
ax.plot(overlaptt,coveigvecseries[:,itrial,1],'g',label='2 cut off')
# ax.plot(overlap,eigenvaluechange,'b',label='2 cut off')
# # ax.plot(overlapns,coveigvecseries[:,itrial,2],'c',label='2 cut off')
# ax.plot(overlaptt,eigenvaluechange,'g',label='2 cut off')
ax.plot(overlap,overlap,'r',label='2 cut off')
# ax.set_xlim(xlims)
# ax.set_ylim(ylims)
# ax.set_xticks(xticks)
# ax.set_yticks(yticks)
# ax.set_aspect('equal')

# '''
# individual perturbation of left and right eigenvectors
# '''
# left_theoperturb=y0.T@y0/4/(1-etaseries**2)*gaverage**2
# left_numperturb=statesLeig[:,itrial,2,1]
# figl,axl=plt.subplots(figsize=(5,5))  
# axl.plot(left_theoperturb,left_theoperturb,'b',label='2 cut off')


test normal: (1.0000000000000275+1.1424542206996786e-14j)
<ipython-input-115-87d456a3325b>:39: ComplexWarning: Casting complex values to real discards the imaginary part
  Z[i,i]=1/(b1-eigvJ0[0])
<ipython-input-115-87d456a3325b>:65: RuntimeWarning: divide by zero encountered in true_divide
  coeffsqrtY,coeffsqrtYT = (1-sqrt_1meta2)/sqrt_1meta2**2/8,(1+sqrt_1meta2)/sqrt_1meta2**2/8
test normal 2: (1-8.67746995743113e-30j)
test normal 3: (2.504964790737764+2.861807597882365e-14j)
eigenvalues: [2.50496479+0.j]
<ipython-input-115-87d456a3325b>:69: RuntimeWarning: invalid value encountered in add
  Jprimemat=np.sqrt(coeffsqrtYT[i])*gY.copy().T+np.sqrt(coeffsqrtY[i])*gY
<ipython-input-115-87d456a3325b>:70: RuntimeWarning: invalid value encountered in matmul
  overlaptt[i]=np.squeeze(y0.T@Jprimemat@S0@S0@Jprimemat@x0)
<ipython-input-115-87d456a3325b>:71: RuntimeWarning: invalid value encountered in matmul
  overlapns[i]=np.squeeze(y0.T@Jprimemat@Jprimemat@x0)/eigvAm[0]**2
<ipython-input-115-8

## <l',r'> reflects the direction of changing

In [128]:
'''
variance of r and variance of l <r.Tr>, <l.T,l>
overlap between r and l <l.Tr>
'''
xticks = np.linspace(-0.01,0.1,2)
xlims = [-0.01,0.1]
yticks = np.linspace(-0.01,0.1,2)
ylims = [-0.01,0.1]


itrial=14
gY=np.squeeze(gYbasexee[itrial,:,:])


X0=gY.copy()
J0=Am.copy()+X0.copy()
''' # calculate the original \bar{m} and \bar{n} '''
eigvJ0,eigrvecJ0=la.eig(J0)
inveigrvecJ0=la.inv(eigrvecJ0)
meig0 = np.squeeze(eigrvecJ0[:,:].copy())
neig0 = np.squeeze(inveigrvecJ0[:,:].copy()) # inverse
neig0=neig0.copy().T
print('test normal:',np.sum(neig0[:,0]*meig0[:,0]))
for i in range(nrank):
    neig0[:,i]*=eigvJ0[i]
for j in range(1,N):
    neig0[:,j]*=eigvJ0[j]

leig0, reig0 = np.zeros((N,N)),np.zeros((N,N))
''' The first '''
reig0=meig0.copy()
normval=np.sum(reig0*neig0.copy(),axis=0)
normval=np.repeat(np.reshape(normval,(1,N)),N,axis=0)
leig0=neig0.copy()/normval.copy()
''' calculate s0'''
Z = np.zeros((NE*2-1,NE*2-1))
for i in range(NE*2-1):
    b1=eigvJ0[i+1]
    Z[i,i]=1/(b1-eigvJ0[0])
S0=reig0[:,1:]@Z@leig0[:,1:].T
print('test normal 2:',np.sum(leig0[:,0]*reig0[:,0]))
print('test normal 3:',np.sum(meig0[:,0]*neig0[:,0]))
print('eigenvalues:',eigvJ0[:nrank])
'''## for reference '''
x0,y0=np.reshape(reig0[:,0],(NE*2,1)),np.reshape(leig0[:,0].copy(),(NE*2,1))
''' ## normalization 0 (reference) absolute reference is xAm, yAm '''
tildex,tildey=np.reshape(x0[:,0].copy(),(NE*2,1)),np.reshape(y0[:,0].copy(),(NE*2,1))
## make sure the E in x is positive, if negative then all x and y should change sign
if (np.mean(tildex[:NE])*xAm[0,0])<0:
    tildex*=(-1)
    tildey*=(-1)
x0,y0=np.reshape(tildex,(NE*2,1)),np.reshape(tildey,(NE*2,1))
x0=np.sqrt(np.squeeze(tildey.T@xAm)/np.squeeze(yAm.T@tildex))*tildex
y0=np.sqrt(np.squeeze(tildex.T@yAm)/np.squeeze(xAm.T@tildey))*tildey
x0,y0=np.reshape(x0,(NE*2,1)),np.reshape(y0,(NE*2,1))

'''
rho_prime=eta/(2*sqrt(1-eta**2))
rho'/2/sqrt(rho) to 2nd=(1+np.sqrt(1-etaseries**2))/8/(1-etaseries**2)
rho'/2/sqrt(1-rho) to 2nd=(1-np.sqrt(1-etaseries**2))/8/(1-etaseries**2)
'''
heta=etaseries[1]-etaseries[0]
rhoseries=(1-np.sqrt(1-etaseries**2))/2.0 ## when eta = 0, asqr = 0, aamp = 0, XT-0, X-1
sqrt_1meta2= np.sqrt(1-etaseries**2)
coeffsqrtY,coeffsqrtYT = (1-sqrt_1meta2)/sqrt_1meta2**2/8,(1+sqrt_1meta2)/sqrt_1meta2**2/8
overlapperturb,overlapvector = np.zeros((neta,2,2,NE),dtype=complex),np.zeros((neta,2,2,NE),dtype=complex)
# EI,LR,:
for i in range(1,neta-1):
    Jprimemat=np.sqrt(coeffsqrtYT[i])*gY.copy().T-np.sqrt(coeffsqrtY[i])*gY
    leftperturb=np.squeeze(-np.squeeze(y0.T@Jprimemat))/(-eigvAm[0])# left perturbation E
    rightperturb=np.squeeze(-np.squeeze(Jprimemat@x0))/(-eigvAm[0])#*Beigvseries[i,itrial,0]# right perturbation E
    overlapperturb[i,0,0,:],overlapperturb[i,0,1,:]=leftperturb[:NE],rightperturb[:NE]
    overlapperturb[i,1,0,:],overlapperturb[i,1,1,:]=leftperturb[NE:],rightperturb[NE:]
    overlapvector[i,0,0,:]=np.squeeze(Leigvecseries[i,itrial,:NE,0])-np.mean(np.squeeze(Leigvecseries[i,itrial,:NE,0]))
    overlapvector[i,1,0,:]=np.squeeze(Leigvecseries[i,itrial,NE:,0])-np.mean(np.squeeze(Leigvecseries[i,itrial,NE:,0]))
    overlapvector[i,0,1,:]=np.squeeze(Reigvecseries[i,itrial,:NE,0])-np.mean(np.squeeze(Reigvecseries[i,itrial,:NE,0]))
    overlapvector[i,1,1,:]=np.squeeze(Reigvecseries[i,itrial,NE:,0])-np.mean(np.squeeze(Reigvecseries[i,itrial,NE:,0]))
## E population 
# vectorcovE,perturbationcovE = np.zeros((neta,2,2)),np.zeros((neta,2,2))
directionvecE,directionvecI,directionperturbE,directionperturbI=np.zeros((neta,2,2)),np.zeros((neta,2,2)),np.zeros((neta,2,2)),np.zeros((neta,2,2))
ampvecE,ampvecI,ampperturbE,ampperturbI=np.zeros((neta,2)),np.zeros((neta,2)),np.zeros((neta,2)),np.zeros((neta,2))
for i in range(neta):
    vectorEmat,vectorImat=np.squeeze(overlapvector[i,0,:,:]),np.squeeze(overlapvector[i,1,:,:])
    perturbEmat,perturbImat=np.squeeze(overlapperturb[i,0,:,:]),np.squeeze(overlapperturb[i,1,:,:])
    vcovE,vcovI=vectorEmat@vectorEmat.T,vectorImat@vectorImat.T
    pcovE,pcovI=perturbEmat@perturbEmat.T ,perturbImat@perturbImat.T

    vvE,dirvE=la.eig(vcovE)
    vvI,dirvI=la.eig(vcovI)
    vpE,dirpE=la.eig(pcovE)
    vpI,dirpI=la.eig(pcovI)
    directionvecE[i,:,:],directionvecI[i,:,:]=dirvE,dirvI
    directionperturbE[i,:,:],directionperturbI[i,:,:]=dirpE,dirpI
    ampvecE[i,:],ampvecI[i,:]=vvE,vvI
    ampperturbE[i,:],ampperturbI[i,:]=vpE,vpI



# confidence_ellipse(Leftperturb,Rightperturb,ax,edgecolor='orange')
# confidence_ellipse(np.real(nIvec),np.real(mIvec),ax,edgecolor='green')



test normal: (0.9999999999999927-1.8676725818116555e-15j)
<ipython-input-128-8665c9879596>:39: ComplexWarning: Casting complex values to real discards the imaginary part
  Z[i,i]=1/(b1-eigvJ0[0])
<ipython-input-128-8665c9879596>:65: RuntimeWarning: divide by zero encountered in true_divide
  coeffsqrtY,coeffsqrtYT = (1-sqrt_1meta2)/sqrt_1meta2**2/8,(1+sqrt_1meta2)/sqrt_1meta2**2/8
test normal 2: (0.9999999999999996+9.466330862652142e-30j)
test normal 3: (-3.003105340076279+5.608817503952681e-15j)
eigenvalues: [-3.00310534+0.j]
<ipython-input-128-8665c9879596>:92: ComplexWarning: Casting complex values to real discards the imaginary part
  directionvecE[i,:,:],directionvecI[i,:,:]=dirvE,dirvI
<ipython-input-128-8665c9879596>:93: ComplexWarning: Casting complex values to real discards the imaginary part
  directionperturbE[i,:,:],directionperturbI[i,:,:]=dirpE,dirpI
<ipython-input-128-8665c9879596>:94: ComplexWarning: Casting complex values to real discards the imaginary part
  ampvecE[i

In [143]:

fig,ax=plt.subplots(figsize=(5,5)) 
for idxeta in np.arange(1,8):
    heta=0.1
    ax.plot([0,0+ampvecE[idxeta,0]*directionvecE[idxeta,0,0]],[0,0+ampvecE[idxeta,0]*directionvecE[idxeta,1,0]],color='r',linestyle='-',linewidth=1.5)
    ax.plot([0,0+ampvecI[idxeta,0]*directionvecI[idxeta,0,0]],[0,0+ampvecI[idxeta,0]*directionvecI[idxeta,1,0]],color='orange',linestyle='-',linewidth=1.5)
    # ax.plot([0,0+ampvecE[idxeta+1,0]*directionvecE[idxeta+1,0,0]],[0,0+ampvecE[idxeta+1,0]*directionvecE[idxeta+1,1,0]],color='r',linestyle='-',linewidth=1.5)
    # ax.plot([0,0+ampvecI[idxeta+1,0]*directionvecI[idxeta+1,0,0]],[0,0+ampvecI[idxeta+1,0]*directionvecI[idxeta+1,1,0]],color='orange',linestyle='-',linewidth=1.5)

    vecEx,vecEy=ampvecE[idxeta,0]*directionvecE[idxeta,0,0],ampvecE[idxeta,0]*directionvecE[idxeta,1,0]
    vecIx,vecIy=ampvecI[idxeta,0]*directionvecI[idxeta,0,0],ampvecI[idxeta,0]*directionvecI[idxeta,1,0]
    ax.plot([vecEx,vecEx+ampperturbE[idxeta,1]*directionperturbE[idxeta,0,1]*heta],[vecEy,vecEy+ampperturbE[idxeta,1]*directionperturbE[idxeta,1,1]*heta],color='b',linestyle='--',linewidth=1.5)
    ax.plot([vecIx,vecIx+ampperturbI[idxeta,1]*directionperturbI[idxeta,0,1]*heta],[vecIy,vecIy+ampperturbI[idxeta,1]*directionperturbI[idxeta,1,1]*heta],color='g',linestyle='--',linewidth=1.5)

xticks = np.linspace(-0.0,0.15,2)
xlims = [-0.00,0.15]
yticks = np.linspace(-0.0,0.035,2)
ylims = [-0.0,0.025]
ax.set_xlim(xlims)
ax.set_ylim(ylims)
ax.set_xticks(xticks)
ax.set_yticks(yticks)
# ax.set_aspect('equal')



# fig,ax=plt.subplots(figsize=(5,5))  
# ax.scatter(overlapperturb[idxeta,0,0,:],overlapperturb[idxeta,0,1,:],s=5,c='orange',alpha=0.5)
# ax.scatter(Leigvecseries[idxeta,itrial,:NE,0]-np.mean(Leigvecseries[idxeta,itrial,:NE,0]),Reigvecseries[idxeta,itrial,:NE,0]-np.mean(Reigvecseries[idxeta,itrial,:NE,0]),s=5,c='blue',alpha=0.5)
# xticks = np.linspace(-0.06,0.06,2)
# xlims = [-0.06,0.06]
# yticks = np.linspace(-0.06,0.06,2)
# ylims = [-0.06,0.06]
# ax.set_xlim(xlims)
# ax.set_ylim(ylims)
# ax.set_xticks(xticks)
# ax.set_yticks(yticks)
# ax.set_aspect('equal')

# fig,ax=plt.subplots(figsize=(5,5))  
# ax.scatter(overlapperturb[idxeta,1,0,:],overlapperturb[idxeta,1,1,:],s=5,c='orange',alpha=0.5)
# ax.scatter(Leigvecseries[idxeta,itrial,NE:,0]-np.mean(Leigvecseries[idxeta,itrial,NE:,0]),Reigvecseries[idxeta,itrial,NE:,0]-np.mean(Reigvecseries[idxeta,itrial,NE:,0]),s=5,c='blue',alpha=0.5)
# ax.set_aspect('equal')
# ax.set_xlim(xlims)
# ax.set_ylim(ylims)
# ax.set_xticks(xticks)
# ax.set_yticks(yticks)
# ax.set_aspect('equal')

In [103]:
h=etaseries[1]-etaseries[0]
fig,ax=plt.subplots(figsize=(5,5))  
ax.scatter(overlapperturb[idxeta,0,0,:]*h,Leigvecseries[idxeta,itrial,:NE,0]-(Leigvecseries[idxeta-1,itrial,:NE,0]),s=5,c='orange',alpha=0.5)
ax.scatter(overlapperturb[idxeta,1,0,:]*h,Leigvecseries[idxeta,itrial,NE:,0]-(Leigvecseries[idxeta-1,itrial,NE:,0]),s=5,c='blue',alpha=0.5)
ax.set_aspect('equal')

fig,ax=plt.subplots(figsize=(5,5))  
ax.scatter(overlapperturb[idxeta,0,1,:]*h,Reigvecseries[idxeta,itrial,:NE,0]-(Reigvecseries[idxeta-1,itrial,:NE,0]),s=5,c='orange',alpha=0.5)
ax.scatter(overlapperturb[idxeta,1,1,:]*h,Reigvecseries[idxeta,itrial,NE:,0]-(Reigvecseries[idxeta-1,itrial,NE:,0]),s=5,c='blue',alpha=0.5)
ax.set_aspect('equal')

In [80]:
'''
individual perturbation of left and right eigenvectors
'''
left_theoperturb=np.sqrt(y0.T@y0/4/(1-etaseries**2)*gaverage**2)/eigvAm[0]**2
left_theoperturb[np.isnan(left_theoperturb)]=0
left_theoperturb=np.squeeze(left_theoperturb)
left_numperturb=(statesLeig[0,itrial,1,0]-statesLeig[:,itrial,1,0])**2
left_numperturb+=(statesLeig[0,itrial,0,0]-statesLeig[:,itrial,0,0])**2
# left_theoperturb=np.zeros(neta,dtype=complex)
# for i in range(1,neta):
#     Jprimemat=coeffsqrtYT[i]*gY.copy().T+coeffsqrtY[i]*gY
#     JTprimemat=coeffsqrtYT[i]*gY.copy()+coeffsqrtY[i]*gY.copy().T
#     left_theoperturb[i]=np.squeeze(y0.T@Jprimemat@S0@S0.T@Jprimemat.T@y0)
# left_numperturb=statesLeig[:,itrial,2,1]
# figl,axl=plt.subplots(figsize=(5,5))  
axl.plot(left_theoperturb,left_numperturb,'b',label='2 cut off')
axl.plot(left_numperturb,left_numperturb,'r',label='2 cut off')
print(left_theoperturb)
print(left_numperturb)

[0.06663197+5.48736562e-17j 0.06704713+5.52155515e-17j
 0.06834076+5.62809015e-17j 0.07067388+5.82023017e-17j
 0.07438211+6.12561545e-17j 0.08013664+6.59952068e-17j
 0.08939617+7.36207353e-17j 0.10601082+8.73034525e-17j
 0.14544564+1.19779349e-16j 0.        +0.00000000e+00j]
[0.00000000e+00+0.00000000e+00j 5.23109940e-09-2.93880565e-19j
 2.01869114e-08-6.42399951e-19j 4.36754503e-08-4.24773095e-19j
 7.43198174e-08-2.20395735e-19j 1.10419616e-07-2.63155313e-19j
 1.49677640e-07-4.64674326e-19j 1.88533030e-07-5.98614840e-19j
 2.19837821e-07+3.17417768e-19j 1.93688017e-07+1.51313930e-19j]
<ipython-input-80-53250e99f466>:4: RuntimeWarning: divide by zero encountered in true_divide
  left_theoperturb=np.sqrt(y0.T@y0/4/(1-etaseries**2)*gaverage**2)/eigvAm[0]**2
<ipython-input-80-53250e99f466>:4: RuntimeWarning: invalid value encountered in multiply
  left_theoperturb=np.sqrt(y0.T@y0/4/(1-etaseries**2)*gaverage**2)/eigvAm[0]**2
/Applications/Anaconda/anaconda3/lib/python3.8/site-packages/numpy

# the flow - 
## from unperturbed connectivity $\to$ perturbed connectivity without symmetry $\to$ perturbed connectivity with symmetry

In [239]:
'''
the whole changing process
from unperturbed connectivity to perturbed connectivity without symmetry to perturbed connectivity with symmetry
### >>>>>>>>> something to be noted 
    * use identical 
'''
# generate mean matrix
nrank=1
Nt=np.array([1000,1000])
NE,NI=Nt[0],Nt[1]
N=NE+NI
Nparams=np.array([NE,NI])
nrank,ntrial,neta,ngavg=1,20,1,1
## for all casesq
xticks = np.linspace(-4.0,3.0,5)
xlims = [-4.1,3.1]
yticks = np.linspace(-1.3,1.3,3)
ylims = [-1.31,1.31]
theta = np.linspace(0, 2 * np.pi, 200)

''' C0 rules '''
Xtest = np.zeros((NE*2,NE*2))
Xtest[:NE,:NE]=1
idxEE,idyEE=np.where(Xtest>0)
lowinEE=np.where(idxEE<idyEE)
upinEE=np.where(idxEE>idyEE)
idxUpEE,idyUpEE,idxDownEE,idyDownEE=idxEE[lowinEE],idyEE[lowinEE],idxEE[upinEE],idyEE[upinEE]
Xrep = np.zeros_like(Xtest)
Xrep[idxUpEE,idyUpEE]=1
idxUpII,idyUpII,idxDownII,idyDownII=idxUpEE+NE,idyUpEE+NE,idxDownEE+NE,idyDownEE+NE

'''
## parameters for unperturbed connectivity
'''
JI,JE,a,b=3.50,1.0,-0.6,0.6 # already recording -3.5# 2.0
a,b=0.0,0.0
# JI,JE,a,b=3.5,1.0,3.5,1.0 # already recording -3.5# 2.0
# JI,JE=0,0
JEE,JIE,JEI,JII=JE+a,JE-a,JI-b,JI+b
'''
Am -- J(g0=0), l0(g0=0), r0(g0=0), S=R1(B1-lambda0In-1)^(-1)L1.T
'''
Am,Jsv=generate_meanmat_eig(Nparams,JEE,JIE,JEI,JII)
# calculate the original \bar{m} and \bar{n}
eigvAm,eigveclAm,eigvecrAm=scpla.eig(Am,left=True,right=True)
meigvecAm=eigvecrAm.copy()
neigvecAm=eigveclAm.copy()
for i in range(nrank):
    normalizecoe=(np.sum(neigvecAm[:,i]*meigvecAm[:,i]))
    neigvecAm[:,i]*=(eigvAm[i]/normalizecoe)
for i in range(1,NE*2):
    normalizecoe=(np.sum(neigvecAm[:,i]*meigvecAm[:,i]))
    neigvecAm[:,i]/=normalizecoe
print('normalization m & n')
print(la.norm(meigvecAm[:,0]),la.norm(neigvecAm[:,0]))
print('eigenvalues:',eigvAm[:nrank])
'''
for reference
'''
xAm,yAm=np.reshape(meigvecAm[:,0],(NE*2,1)),np.reshape(neigvecAm[:,0].copy()/eigvAm[0],(NE*2,1))
'''
inverse S
'''
invB1=np.zeros((NE*2-1,NE*2-1))
for i in range(NE*2-1):
    invB1[i,i]=1/(eigvAm[i+1]-eigvAm[0])
X1,Y1=meigvecAm[:,1:].copy(),neigvecAm[:,1:].copy()
SAm = X1@invB1@Y1.T

'''
Network Setting for random gain 
'''
signeta=np.ones(3)
# signeta[1]*=(-1)
## heterogeneous random gain
'''
>>>>>>>>>>>>>> Case 1 and Case 2 >>>>>>>>>>>>>>>>>>>>>>>
'''
xee,xei,xie,xii=1.0,1.0,1.0,1.0 ## HOMOgeneous random gain g_kl
# xee,xei,xie,xii=1.0,0.5,0.2,0.8 ## HETEROgeneous random gain
# xee,xei,xie,xii=1.0,0.5,0.3,0.1

coeffeta=np.array([1.0,1.0,1.0])
'''
Recording Variables
Note>>>>>
    * the first '2' in each ndarray means the first stage -- without symmetry and the second stage -- with symmetry
    * the same trial share the same gY -- in the case with symmetric conn. gY and gY.T
'''
# original data
eigvalseries,mvecseries,nvecseries=np.zeros((2,ntrial,nrank),dtype=complex),np.zeros((2,ntrial,NE*2,nrank),dtype=complex),np.zeros((2,ntrial,NE*2,nrank),dtype=complex)
Reigvecseries,Leigvecseries=np.zeros((2,ntrial,NE*2,N),dtype=complex),np.zeros((2,ntrial,NE*2,N),dtype=complex)
Beigvseries=np.zeros((2,ntrial,N),dtype=complex)
''' All recorded connectivity statistics '''
statesReig,statesLeig=np.zeros((2,ntrial,3,2),dtype=complex),np.zeros((2,ntrial,3,2),dtype=complex) #E,I,Total/mean,std# 
coveigvecseries=np.zeros((2,ntrial,3),dtype=complex) # E, I and total
''' All random matrices for each trials '''
## Not X total (symmetry), but Y independent
gYbasexee=np.zeros((ntrial,NE*2,NE*2))
'''
for each individual trial, supposed to continuously increase g from zero to one -- see the sensibility changing
but within one single trial, the random matrix (also random structure) keeps unchanged, thus avoid changing in random pattern
'''

''' Iterative Processing '''
gaverage,eta=1.0,0.8
for iktrial in range(ntrial):
    
    # >>> generate the (base/ref) random matrix for each trial
    Xsym=np.reshape(np.random.randn(NE*2*NE*2),(NE*2,NE*2))/np.sqrt(NE)
    Xsym[:NE,:NE]=xee*gaverage*np.reshape(np.random.randn(NE*NE),(NE,NE))/np.sqrt(2*NE)
    Xsym[NE:,NE:]=xii*gaverage*np.reshape(np.random.randn(NE*NE),(NE,NE))/np.sqrt(2*NE)
    Xsym[NE:,:NE]=xie*gaverage*np.reshape(np.random.randn(NE*NE),(NE,NE))/np.sqrt(2*NE)
    Xsym[:NE,NE:]=xei*gaverage*np.reshape(np.random.randn(NE*NE),(NE,NE))/np.sqrt(2*NE)
    XsymT=Xsym.copy().T


    ''' >>>>>>>>>>>>>>>>> without symmetry >>>>>>>>>>  '''
    X=Xsym.copy()
    idxeta = 0 ## the first stage -- without symmetry
    # overall
    J = X.copy()+Am.copy()
    JT = J.copy().T
    symnumfull=np.sum(np.sum(J*JT))/(4*NE**2)-np.mean(np.mean(J.copy()))**2
    symnumfull*=2*NE
    print('eta full:',symtheofull,'; num (should be zero):',symnumfull)
    '''
    statistics: Full connectivity
    '''
    # properties
    eigvJ,eigrvecJ=la.eig(J)
    inveigrvecJ=la.inv(eigrvecJ)
    meig = np.squeeze(eigrvecJ[:,:].copy())
    # meig = np.reshape(meig,(NE*2,nrank))
    neig = np.squeeze(inveigrvecJ[:,:].copy()) # inverse
    neig=neig.copy().T
    # neig = np.reshape(neig.T,(NE*2,nrank))
    for i in range(nrank):
        neig[:,i]*=eigvJ[i]
    for j in range(1,N):
        neig[:,j]*=eigvJ[j]
    ''' low rank properties m,n,eigenvalues '''
    eigvalseries[idxeta,iktrial,0],mvecseries[idxeta,iktrial,:,0],nvecseries[idxeta,iktrial,:,0]=eigvJ[0],meig[:,0].copy(),neig[:,0].copy() 
    
    Leig, Reig = np.zeros((N,N)),np.zeros((N,N))
    ''' The first '''
    Reig=meig.copy()
    normval=np.sum(Reig*neig.copy(),axis=0)
    normval=np.repeat(np.reshape(normval,(1,N)),N,axis=0)
    ''' eigendecomposition properties left and right eigenvectors '''
    Leig=neig.copy()/normval.copy() 
    '''
    normalization 0 (reference)
    '''
    tildex,tildey=np.reshape(Reig[:,0].copy(),(NE*2,1)),np.reshape(Leig[:,0].copy(),(NE*2,1))
    if (np.mean(tildex[:NE])*xAm[0,0])<0:
        tildex*=(-1)
        tildey*=(-1)
    xnorm0=np.sqrt(np.squeeze(tildey.T@x0)/np.squeeze(y0.T@tildex))*tildex
    ynorm0=np.sqrt(np.squeeze(tildex.T@y0)/np.squeeze(x0.T@tildey))*tildey
    ## >>>>>>> '0' --> for the first stage without symmetr
    Reigvecseries[idxeta,iktrial,:,:],Leigvecseries[idxeta,iktrial,:,:]=Reig.copy(),Leig.copy()
    Beigvseries[idxeta,iktrial,:]=eigvJ.copy()
    Reigvecseries[idxeta,iktrial,:,0],Leigvecseries[idxeta,iktrial,:,0]=xnorm0[:,0].copy(),ynorm0[:,0].copy()
    '''  RECORDING STATES 
    * statesReig/Leig[grandom,trial,E/I/Total,mean/std]

    '''
    statesReig[idxeta,iktrial,0,0],statesReig[idxeta,iktrial,1,0]=np.mean(Reigvecseries[idxeta,iktrial,:NE,0]),np.mean(Reigvecseries[idxeta,iktrial,NE:,0])
    statesReig[idxeta,iktrial,2,0]=np.mean(Reigvecseries[idxeta,iktrial,:,0])
    statesReig[idxeta,iktrial,0,1],statesReig[idxeta,iktrial,1,1]=np.std(Reigvecseries[idxeta,iktrial,:NE,0]),np.std(Reigvecseries[idxeta,iktrial,NE:,0])
    statesReig[idxeta,iktrial,2,1]=np.std(Reigvecseries[idxeta,iktrial,:,0])
    
    statesLeig[idxeta,iktrial,0,0],statesLeig[idxeta,iktrial,1,0]=np.mean(Leigvecseries[idxeta,iktrial,:NE,0]),np.mean(Leigvecseries[idxeta,iktrial,NE:,0])
    statesLeig[idxeta,iktrial,2,0]=np.mean(Leigvecseries[idxeta,iktrial,:,0])
    statesLeig[idxeta,iktrial,0,1],statesLeig[idxeta,iktrial,1,1]=np.std(Leigvecseries[idxeta,iktrial,:NE,0]),np.std(Leigvecseries[idxeta,iktrial,NE:,0])
    statesLeig[idxeta,iktrial,2,1]=np.std(Leigvecseries[idxeta,iktrial,:,0])
    ''' covariance '''
    '''
    Note, we should change the normalization for the first item to the (Refer) normalization 
    tildeReig,tildeLeig
    normalization 0
    '''
    ## normalization 0 reference
    tildeReig,tildeLeig=np.squeeze(Reigvecseries[idxeta,iktrial,:,:]),np.squeeze(Leigvecseries[idxeta,iktrial,:,:])
    reig,leig=np.reshape(tildeReig[:,0].copy(),(NE*2,1)),np.reshape(tildeLeig[:,0].copy(),(NE*2,1))
    ## mean at eact point
    rEceneig,rIceneig = np.mean(reig[:NE,0]),np.mean(reig[NE:,0])
    lEceneig,lIceneig = np.mean(leig[:NE,0]),np.mean(leig[NE:,0])
    ## random components
    rrandeig,lrandeig=reig.copy(),leig.copy()
    rrandeig[:NE,0],rrandeig[NE:,0]=rrandeig[:NE,0]-rEceneig,rrandeig[NE:,0]-rIceneig
    lrandeig[:NE,0],lrandeig[NE:,0]=lrandeig[:NE,0]-lEceneig,lrandeig[NE:,0]-lIceneig

    coveigvecseries[idxeta,iktrial,0]=np.reshape(rrandeig[:NE,0],(NE,1)).T@np.reshape(lrandeig[:NE,0],(NE,1))
    coveigvecseries[idxeta,iktrial,1]=np.reshape(rrandeig[NE:,0],(NE,1)).T@np.reshape(lrandeig[NE:,0],(NE,1))
    coveigvecseries[idxeta,iktrial,2]=coveigvecseries[idxeta,iktrial,0]+coveigvecseries[idxeta,iktrial,1]

    '''
    converting initial x0 and y0 to the one with random components
    '''
    x0,y0=np.reshape(np.squeeze(Reigvecseries[idxeta,iktrial,:,0]),(NE*2,1)),np.reshape(np.squeeze(Leigvecseries[idxeta,iktrial,:,0]),(2*NE,1))

    ''' >>>>>>>>>>>>>>>>> with symmetry >>>>>>>>>>  '''
    # etaset[0]*=0.1
    Xinit = Xsym.copy()
    idxeta=1
    etaset=eta*np.ones(3)
    etaset=etaset*coeffeta
    print('a set of symmetry:',etaset)
    ''' heterogeneous ETA method 1 '''
    ## EE ##
    asqr=(1-np.sqrt(1-etaset[0]**2))/2.0 ## when eta = 0, asqr = 0, aamp = 0, XT-0, X-1
    aamp=np.sqrt(asqr)
    Xinit[:NE,:NE]=signeta[0]*aamp*XsymT[:NE,:NE].copy()+np.sqrt(1-aamp**2)*Xsym[:NE,:NE].copy()
    ## EI IE##
    asqr=(1-np.sqrt(1-etaset[1]**2))/2.0
    aamp=np.sqrt(asqr)
    Xinit[NE:,:NE]=signeta[1]*aamp*XsymT[NE:,:NE].copy()+np.sqrt(1-aamp**2)*Xsym[NE:,:NE].copy()
    Xinit[:NE,NE:]=signeta[1]*aamp*XsymT[:NE,NE:].copy()+np.sqrt(1-aamp**2)*Xsym[:NE,NE:].copy()
    ## II ##
    asqr=(1-np.sqrt(1-etaset[2]**2))/2.0
    aamp=np.sqrt(asqr)
    Xinit[NE:,NE:]=signeta[2]*aamp*XsymT[NE:,NE:].copy()+np.sqrt(1-aamp**2)*Xsym[NE:,NE:].copy()
    ''' THEORETICAL FULL ETA '''
    symtheofull=gaverage**2*(etaset[0]*signeta[0]+etaset[2]*signeta[2]+2*etaset[1]*signeta[1])/4.0#+(a-b)**2/(NE*2)
    X=Xinit.copy()

    # overall
    J = X.copy()+Am.copy()
    JT = J.copy().T
    symnumfull=np.sum(np.sum(J*JT))/(4*NE**2)-np.mean(np.mean(J.copy()))**2
    symnumfull*=2*NE
    print('compare degree of symmetry:',symnumfull,'--num; ',etaset*signeta,'--setting')
    # properties
    eigvJ,eigrvecJ=la.eig(J)
    inveigrvecJ=la.inv(eigrvecJ)
    meig = np.squeeze(eigrvecJ[:,:].copy())
    # meig = np.reshape(meig,(NE*2,nrank))
    neig = np.squeeze(inveigrvecJ[:,:].copy()) # inverse
    neig=neig.copy().T
    print('test normal 4:',np.sum(neig[:,0]*meig[:,0]))
    for i in range(nrank):
        neig[:,i]*=eigvJ[i]
    for j in range(1,N):
        neig[:,j]*=eigvJ[j]
    ## >>> for low rank framework m and n
    eigvalseries[idxeta,iktrial,0],mvecseries[idxeta,iktrial,:,0],nvecseries[idxeta,iktrial,:,0]=eigvJ[0],meig[:,0].copy(),neig[:,0].copy()
    
    ## >>> for eigendecomposition 
    Leig, Reig = np.zeros((N,N)),np.zeros((N,N))
    ''' The first '''
    Reig=meig.copy()
    normval=np.sum(Reig*neig.copy(),axis=0)
    normval=np.repeat(np.reshape(normval,(1,N)),N,axis=0)
    Leig=neig.copy()/normval.copy()
    print('test normal 5:',np.sum(Leig[:,0]*Reig[:,0]))
    '''
    normalization 0 (reference)
    '''
    tildex,tildey=Reig[:,0].copy(),Leig[:,0].copy()
    if (np.mean(tildex[:NE])*xAm[0,0])<0:
        tildex*=(-1)
        tildey*=(-1)
    xnorm0=np.sqrt(np.squeeze(tildey.T@x0)/np.squeeze(y0.T@tildex))*tildex
    ynorm0=np.sqrt(np.squeeze(tildex.T@y0)/np.squeeze(x0.T@tildey))*tildey

    Reigvecseries[idxeta,iktrial,:,:],Leigvecseries[idxeta,iktrial,:,:]=Reig.copy(),Leig.copy()
    Beigvseries[idxeta,iktrial,:]=eigvJ.copy()
    Reigvecseries[idxeta,iktrial,:,0],Leigvecseries[idxeta,iktrial,:,0]=xnorm0[:].copy(),ynorm0[:].copy()

    '''  RECORDING STATES '''
    statesReig[idxeta,iktrial,0,0],statesReig[idxeta,iktrial,1,0]=np.mean(Reigvecseries[idxeta,iktrial,:NE,0]),np.mean(Reigvecseries[idxeta,iktrial,NE:,0])
    statesReig[idxeta,iktrial,2,0]=np.mean(Reigvecseries[idxeta,iktrial,:,0])
    statesReig[idxeta,iktrial,0,1],statesReig[idxeta,iktrial,1,1]=np.std(Reigvecseries[idxeta,iktrial,:NE,0]),np.std(Reigvecseries[idxeta,iktrial,NE:,0])
    statesReig[idxeta,iktrial,2,1]=np.std(Reigvecseries[idxeta,iktrial,:,0])
    
    statesLeig[idxeta,iktrial,0,0],statesLeig[idxeta,iktrial,1,0]=np.mean(Leigvecseries[idxeta,iktrial,:NE,0]),np.mean(Leigvecseries[idxeta,iktrial,NE:,0])
    statesLeig[idxeta,iktrial,2,0]=np.mean(Leigvecseries[idxeta,iktrial,:,0])
    statesLeig[idxeta,iktrial,0,1],statesLeig[idxeta,iktrial,1,1]=np.std(Leigvecseries[idxeta,iktrial,:NE,0]),np.std(Leigvecseries[idxeta,iktrial,NE:,0])
    statesLeig[idxeta,iktrial,2,1]=np.std(Leigvecseries[idxeta,iktrial,:,0])

    ''' covariance '''
    '''
    Note, we should change the normalization for the first item to the (Refer) normalization 
    tildeReig,tildeLeig
    normalization 0
    '''
    ## normalization 0 reference
    tildeReig,tildeLeig=np.squeeze(Reigvecseries[idxeta,iktrial,:,:]),np.squeeze(Leigvecseries[idxeta,iktrial,:,:])
    reig,leig=np.reshape(tildeReig[:,0].copy(),(NE*2,1)),np.reshape(tildeLeig[:,0].copy(),(NE*2,1))
    ## mean at eact point
    rEceneig,rIceneig = np.mean(reig[:NE,0]),np.mean(reig[NE:,0])
    lEceneig,lIceneig = np.mean(leig[:NE,0]),np.mean(leig[NE:,0])
    ## random components
    rrandeig,lrandeig=reig.copy(),leig.copy()
    rrandeig[:NE,0],rrandeig[NE:,0]=rrandeig[:NE,0]-rEceneig,rrandeig[NE:,0]-rIceneig
    lrandeig[:NE,0],lrandeig[NE:,0]=lrandeig[:NE,0]-lEceneig,lrandeig[NE:,0]-lIceneig

    coveigvecseries[idxeta,iktrial,0]=np.reshape(rrandeig[:NE,0],(NE,1)).T@np.reshape(lrandeig[:NE,0],(NE,1))
    coveigvecseries[idxeta,iktrial,1]=np.reshape(rrandeig[NE:,0],(NE,1)).T@np.reshape(lrandeig[NE:,0],(NE,1))
    coveigvecseries[idxeta,iktrial,2]=coveigvecseries[idxeta,iktrial,0]+coveigvecseries[idxeta,iktrial,1]
    gYbasexee[iktrial,:,:]=Xsym.copy()


normalization m & n
1.0000000000000004 5.147815070493496
eigenvalues: [-2.5+0.j]
<ipython-input-239-2ccc2449dc8d>:66: ComplexWarning: Casting complex values to real discards the imaginary part
  invB1[i,i]=1/(eigvAm[i+1]-eigvAm[0])
eta full: 0.5120000000000001 ; num (should be zero): 0.0012352829008212946
a set of symmetry: [0.8 0.8 0.8]
compare degree of symmetry: 0.8019463539942836 --num;  [0.8 0.8 0.8] --setting
test normal 4: (0.9999999999999871-5.7620611091761115e-15j)
test normal 5: (1.0000000000000002-2.129924444096732e-29j)
eta full: 0.8 ; num (should be zero): 0.00028387078427365587
a set of symmetry: [0.8 0.8 0.8]
compare degree of symmetry: 0.7996727560060796 --num;  [0.8 0.8 0.8] --setting
test normal 4: (1.0000000000000349+2.1236556030047215e-14j)
test normal 5: (1.0000000000000002-4.733165431326071e-30j)
eta full: 0.8 ; num (should be zero): 0.0004935937496120977
a set of symmetry: [0.8 0.8 0.8]
compare degree of symmetry: 0.7996029574928493 --num;  [0.8 0.8 0.8] --settin

In [260]:
'''
based on ensemble-averaging, ntrial * 
'''
## for all casesq
xticks = np.linspace(-0.15,0.20,2)
xlims = [-0.15,0.20]
yticks = np.linspace(-0.05,0.1,2)
ylims = [-0.05,0.1]

nsymsetE,nsymsetI,symsetE,symsetI=np.zeros((2,ntrial*NE)),np.zeros((2,ntrial*NE)),np.zeros((2,ntrial*NE)),np.zeros((2,ntrial*NE))
nsymsetE[0,:],nsymsetE[1,:]=np.squeeze(Leigvecseries[0,:,:NE,0]).flatten(),np.squeeze(Reigvecseries[0,:,:NE,0]).flatten()
nsymsetI[0,:],nsymsetI[1,:]=np.squeeze(Leigvecseries[0,:,NE:,0]).flatten(),np.squeeze(Reigvecseries[0,:,NE:,0]).flatten()
symsetE[0,:],symsetE[1,:]=np.squeeze(Leigvecseries[1,:,:NE,0]).flatten(),np.squeeze(Reigvecseries[1,:,:NE,0]).flatten()
symsetI[0,:],symsetI[1,:]=np.squeeze(Leigvecseries[1,:,NE:,0]).flatten(),np.squeeze(Reigvecseries[1,:,NE:,0]).flatten()

###  CALCULATE MEAN AND RANDOM 
mnsymE,mnsymI = np.reshape(np.mean(nsymsetE,axis=1),(2,1)),np.reshape(np.mean(nsymsetI,axis=1),(2,1))
msymE,msymI   = np.reshape(np.mean(symsetE,axis=1),(2,1)),np.reshape(np.mean(symsetI,axis=1),(2,1))
rnsymE,rnsymI = nsymsetE-np.repeat(mnsymE,NE*ntrial,axis=1),nsymsetI-np.repeat(mnsymI,NE*ntrial,axis=1)
rsymE,rsymI = symsetE-np.repeat(msymE,NE*ntrial,axis=1),symsetI-np.repeat(msymI,NE*ntrial,axis=1)
### CALCULATE COVARIANCE
covnsymE,covnsymI = rnsymE@rnsymE.T,rnsymI@rnsymI.T
covsymE,covsymI   = rsymE@rsymE.T,rsymI@rsymI.T 
### DIRECTIONS AND EIGENVALUES(A,PLITUDES)
ampnsymE,dirnsymE = la.eig(covnsymE)
ampnsymI,dirnsymI = la.eig(covnsymI)
ampsymE,dirsymE   = la.eig(covsymE)
ampsymI,dirsymI   = la.eig(covsymI)

fig,ax=plt.subplots(figsize=(5,3))  
cm='rb'
# cm='mc'
ax.scatter(nsymsetE[0,:],nsymsetE[1,:],s=5.0,c=cm[0],alpha=0.25)
ax.scatter(nsymsetI[0,:],nsymsetI[1,:],s=5.0,c=cm[1],alpha=0.25)
ax.plot([mnsymE[0],mnsymE[0]+dirnsymE[0,0]],[mnsymE[1],mnsymE[1]+dirnsymE[1,0]],color='grey',linestyle='--',linewidth=1.5)
ax.plot([mnsymE[0],mnsymE[0]+dirnsymE[0,1]],[mnsymE[1],mnsymE[1]+dirnsymE[1,1]],color='grey',linestyle='--',linewidth=1.5)
ax.plot([mnsymI[0],mnsymI[0]+dirnsymI[0,0]],[mnsymI[1],mnsymI[1]+dirnsymI[1,0]],color='grey',linestyle='--',linewidth=1.5)
ax.plot([mnsymI[0],mnsymI[0]+dirnsymI[0,1]],[mnsymI[1],mnsymI[1]+dirnsymI[1,1]],color='grey',linestyle='--',linewidth=1.5)
# cm='rb'
cm='mc'
ax.scatter(symsetE[0,:],symsetE[1,:],s=5.0,c=cm[0],alpha=0.05)
ax.scatter(symsetI[0,:],symsetI[1,:],s=5.0,c=cm[1],alpha=0.05)
ax.plot([msymE[0],msymE[0]+dirsymE[0,0]],[msymE[1],msymE[1]+dirsymE[1,0]],color='grey',linestyle='-.',linewidth=1.5)
ax.plot([msymE[0],msymE[0]+dirsymE[0,1]],[msymE[1],msymE[1]+dirsymE[1,1]],color='grey',linestyle='-.',linewidth=1.5)
ax.plot([msymI[0],msymI[0]+dirsymI[0,0]],[msymI[1],msymI[1]+dirsymI[1,0]],color='grey',linestyle='-.',linewidth=1.5)
ax.plot([msymI[0],msymI[0]+dirsymI[0,1]],[msymI[1],msymI[1]+dirsymI[1,1]],color='grey',linestyle='-.',linewidth=1.5)
ax.scatter(yAm[:NE,0],xAm[:NE,0],s=30.0,c='black',marker='^',alpha=1.0)
ax.scatter(yAm[NE:,0],xAm[NE:,0],s=30.0,c='black',marker='^',alpha=1.0)
ax.set_xlim(xlims)
ax.set_ylim(ylims)
ax.set_xticks(xticks)
ax.set_yticks(yticks)
ax.set_aspect('equal')


<ipython-input-260-99b55e988b11>:11: ComplexWarning: Casting complex values to real discards the imaginary part
  nsymsetE[0,:],nsymsetE[1,:]=np.squeeze(Leigvecseries[0,:,:NE,0]).flatten(),np.squeeze(Reigvecseries[0,:,:NE,0]).flatten()
<ipython-input-260-99b55e988b11>:12: ComplexWarning: Casting complex values to real discards the imaginary part
  nsymsetI[0,:],nsymsetI[1,:]=np.squeeze(Leigvecseries[0,:,NE:,0]).flatten(),np.squeeze(Reigvecseries[0,:,NE:,0]).flatten()
<ipython-input-260-99b55e988b11>:13: ComplexWarning: Casting complex values to real discards the imaginary part
  symsetE[0,:],symsetE[1,:]=np.squeeze(Leigvecseries[1,:,:NE,0]).flatten(),np.squeeze(Reigvecseries[1,:,:NE,0]).flatten()
<ipython-input-260-99b55e988b11>:14: ComplexWarning: Casting complex values to real discards the imaginary part
  symsetI[0,:],symsetI[1,:]=np.squeeze(Leigvecseries[1,:,NE:,0]).flatten(),np.squeeze(Reigvecseries[1,:,NE:,0]).flatten()


In [261]:
# saving data
now = time.strftime("%Y-%m-%d-%H_%M_%S",time.localtime(time.time()))
Jparameters = np.zeros(4+1)
Jparameters[0],Jparameters[1],Jparameters[2],Jparameters[3]=JEE,JEI,JIE,JII
Jparameters[4]=NE
Randgain=np.zeros(4)
Randgain[0],Randgain[1],Randgain[2],Randgain[3]=xee,xei,xie,xii

np.savez(now+'_perturbationConvert_data',x0=x0,y0=y0,xAm=xAm,yAm=yAm,Jparameters=Jparameters,Randgain=Randgain,gaverage=gaverage,eta=eta,signeta=signeta,eigvalseries=eigvalseries,Reigvecseries=Reigvecseries[:,:,:,0],Leigvecseries=Leigvecseries[:,:,:,0],Beigvseries=Beigvseries[:,:,0],meigvecAm=meigvecAm[:,0],neigvecAm=neigvecAm[:,0],eigvAm=eigvAm,statesReig=statesReig,statesLeig=statesLeig,coveigvecseries=coveigvecseries,gYbasexee=gYbasexee,coeffeta=coeffeta)
